**This iPython notebook consists of analysis scripts associated with the article  "Effects of sequence motifs in the yeast 3′ untranslated region determined from massively-parallel assays of random sequences" by Andrew Savinov, Benjamin M. Brandsen, Brooke E. Angell, Josh T. Cuperus, and Stanley Fields. Please cite this article (see below) if you make use of these scripts.**

Full citation:
<br/>Savinov A, Brandsen BM, Angell BE, Cuperus JT, Fields S (2021) Effects of sequence motifs in the yeast 3′ untranslated region determined from massively-parallel assays of random sequences. *bioRxiv*. doi: https://doi.org/10.1101/2021.03.27.437361.

Input data files for the analyses below can be found in the associated Source Data, located in this figshare repository:
https://figshare.com/articles/dataset/Source_Data_for_Savinov_et_al_2021_3_UTRs/16664143
doi: 10.6084/m9.figshare.16664143

This code was written by Andrew Savinov.

Last updated: July 27, 2021.


In [ ]:
import pandas as pandas
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import random
import Levenshtein as lev
import Bio.SeqUtils
from Bio.SeqUtils import GC
import itertools as itertools
import scipy.stats

In [ ]:
## load N50C library data 
N50C_df_path = "[[DATA_DIRECTORY]]/"\
+ "N50C_data" + ".csv"

In [ ]:
results_df = pandas.DataFrame.from_csv(N50C_df_path, index_col=None)

In [ ]:
results_df['rel_log2_score(rescaled+mean-norm)'] =\
(results_df['log2_score']+abs(results_df['log2_score'].min()))\
 .div( (results_df['log2_score']+abs(results_df['log2_score'].min())).mean() )

In [ ]:
print results_df['log2_score'].min()
print results_df['log2_score'].mean()
print (results_df['log2_score']+abs(results_df['log2_score'].min())).mean()

In [ ]:
## to look at native yeast 3' UTR sequences and prior measurements of their properties
nativeUTRs_df_path = "[[DATA_DIRECTORY]]/"\
+ "Geisberg2014_native-mRNA-half-life_with_sequence.csv" 

nativeUTRs_df = pandas.DataFrame.from_csv(nativeUTRs_df_path, index_col=None)
nativeUTRs_df['UTR'] = nativeUTRs_df['sequence'].astype(str)
nativeUTRs_df['Geisberg/min'] = nativeUTRs_df['Half-Life']

In [ ]:
pandas.set_option('max_colwidth', 500)
#pandas.reset_option('max_colwidth')

In [ ]:
nativeUTRs_df['rel_Geisberg/min(mean-norm)'] = nativeUTRs_df['Geisberg/min']\
.div(nativeUTRs_df['Geisberg/min'].mean())


In [ ]:
# Make FASTA file from dataframe of interest.
# (FASTA file of N50C library sequences is needed for running FIMO analysis, see below)

### pick one of the below options: #######
# (1.) N50C library
UTRs_df = results_df['UTR'] #select this for N50C library sequences (>=5 input reads, not pseudocounted)# 

## (2.) Native yeast 3' UTRs
## UTRs_df = nativeUTRs_df['UTR'] #select this for native yeast sequences 3'UTR data
#########################################

length = UTRs_df.size
counter = 0

### pick one of the below options (select as appropriate matching the above selection) #
# (1.)  N50C version:
FASTA_path = "[[DATA_DIRECTORY]]/"\
+ "N50C_data" + ".fasta"
    
## (2.) Native Yeast 3'UTR seqs version #
## FASTA_path = "[[DATA_DIRECTORY]]/"\
## + "Native_seqs" + ".fasta" 
#######################################################################################

UTRs_df_FASTA = open(FASTA_path,"w+")

for i in range(0,length):
    UTRs_df_FASTA.write(">" + str(counter) + "\n")
    UTRs_df_FASTA.write(UTRs_df[counter] + "\n")
    counter+=1
    
UTRs_df_FASTA.close()


In [ ]:
### set definition of enrichment_df (and add 'seq_ID' column) for certain downstream analyses

enrichment_df = results_df.copy()  #>=5 input reads, not pseudocounted, N50C library

enrichment_df['seq_ID']=enrichment_df.index


In [ ]:
## define additional / alternative dataframe name for the N50C data, for easy reference, etc; used extensively in the below scripts
N50C_df = results_df.copy()


In [ ]:
## load N50EPC library growth selection dataframe:

N50EPC_enr_df_path = "[[DATA_DIRECTORY]]/"\
+ "N50EPC_data.csv" 
N50EPC_enr_df = pandas.DataFrame.from_csv(N50EPC_enr_df_path, index_col=None)
N50EPC_enr_df['seq_ID'] = N50EPC_enr_df.index

In [ ]:
## define additional / alternative dataframe name for the native UTRs half-life data; used extensively in the below scripts
enrichment_df_native = nativeUTRs_df.copy() # Native yeast 3'UTRs dataframe
enrichment_df_native['seq_ID']=enrichment_df_native.index

In [ ]:
## define 'UTRs_df' as desired (optional)
#UTRs_df = results_df['UTR'] ## to look at N50C library (non-pseudocounted, >=5 input reads)
UTRs_df = nativeUTRs_df['UTR'] ## to look at native yeast 3' UTRs data

In [ ]:
CYC1_wt_N50 = str.upper('tcgagtcatgtaattagttatgtcacgcttacattcacgccctcccccca')
print CYC1_wt_N50
print ('N50C native CYC1 3\' UTR variant Enr = '+str(N50C_df[N50C_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]))
print ('N50EPC native CYC1 3\' UTR variant Enr = '+str(N50EPC_enr_df[N50EPC_enr_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]))
print
print ('N50C mean Enr = '+str(np.mean(N50C_df['log2_score'])))
print ('N50EPC mean Enr = '+str(np.mean(N50EPC_enr_df['log2_score'])))
print
N50C_mean_Enr_CYC1_norm = np.mean(N50C_df['log2_score']) - N50C_df[N50C_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]
N50EPC_mean_Enr_CYC1_norm = np.mean(N50EPC_enr_df['log2_score']) - N50EPC_enr_df[N50EPC_enr_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]
print ('N50C mean Enr, CYC1 normalized = '+str(N50C_mean_Enr_CYC1_norm))
print ('N50EPC mean Enr, CYC1 normalized = '+str(N50EPC_mean_Enr_CYC1_norm))


In [ ]:
N50C_df['log2_score_CYC1-norm'] = N50C_df['log2_score'] -\
   N50C_df[N50C_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]
N50C_df.head(2)

In [ ]:
N50EPC_enr_df['log2_score_CYC1-norm'] = N50EPC_enr_df['log2_score'] -\
   N50EPC_enr_df[N50EPC_enr_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]
N50EPC_df.head(2)

In [ ]:
print ('N50C min Enr = '+str(np.min(N50C_df['log2_score'])))
print ('N50C mean Enr = '+str(np.mean(N50C_df['log2_score'])))
print ('N50C max Enr = '+str(np.max(N50C_df['log2_score'])))
print
print ('N50EPC min Enr = '+str(np.min(N50EPC_enr_df['log2_score'])))
print ('N50EPC mean Enr = '+str(np.mean(N50EPC_enr_df['log2_score'])))
print ('N50EPC max Enr = '+str(np.max(N50EPC_enr_df['log2_score'])))
print
print
print ('N50C mean CYC1-norm. Enr = '+str(np.mean(N50C_df['log2_score_CYC1-norm'])))
print ('N50EPC mean CYC1-norm. Enr = '+str(np.mean(N50EPC_enr_df['log2_score_CYC1-norm'])))
print
print ('N50C min CYC1-norm. Enr = '+str(np.min(N50C_df['log2_score_CYC1-norm'])))
print ('N50C mean CYC1-norm. Enr = '+str(np.mean(N50C_df['log2_score_CYC1-norm'])))
print ('N50C max CYC1-norm. Enr = '+str(np.max(N50C_df['log2_score_CYC1-norm'])))
print
print ('N50EPC min CYC1-norm. Enr = '+str(np.min(N50EPC_enr_df['log2_score_CYC1-norm'])))
print ('N50EPC mean CYC1-norm. Enr = '+str(np.mean(N50EPC_enr_df['log2_score_CYC1-norm'])))
print ('N50EPC max CYC1-norm. Enr = '+str(np.max(N50EPC_enr_df['log2_score_CYC1-norm'])))

In [ ]:
######  N50C vs. N50EPC analysis

In [ ]:
import re

def regex_search_start(x):
    try:
        return re.search('AATAAA|AAAAAA', str(x)).start()
    except:
        return np.nan
    
def add_columns(analysisDF):
    analysisDF['(TA)3_pos(first)'] = analysisDF['UTR'].str.find('TATATA')
    analysisDF['AAWAAA_pos(first)'] = analysisDF['UTR'].apply\
    (lambda x: regex_search_start(x))
    analysisDF['GC%'] = analysisDF['UTR'].apply(GC)
    analysisDF['AT%'] = 100 - analysisDF['GC%']

def sem(df):
    return np.std(df) / np.sqrt(df.shape[0])

def print_parameters(analysisDF,name):
    print
    print name + ':'
    print 'N = ' + str(analysisDF.shape[0])
    print 'mean +/- sem = ' + str(np.mean(analysisDF['log2_score'])) + ' +\- ' + str(sem(analysisDF['log2_score']))
    
analysisDF = enrichment_df; add_columns(analysisDF)
print_parameters(analysisDF,'N50C, all')
analysisDF = enrichment_df[enrichment_df['UTR'].str.contains('TATATA')]; add_columns(analysisDF)
print_parameters(analysisDF,'N50C, (TA)3-containing')
bins = np.linspace(-10,10,20)
plt.hist(enrichment_df['log2_score'], bins, alpha=0.5 ,color = 'blue', edgecolor='black',\
         density=True, label='N50C, all')
plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'red', edgecolor='black',\
         density=True, label='N50C cont. (TA)3')
plt.legend(loc='upper left')
plt.show()

analysisDF = N50EPC_enr_df; add_columns(analysisDF)
print_parameters(analysisDF,'N50EPC, all')
analysisDF = N50EPC_enr_df[N50EPC_enr_df['UTR'].str.contains('TATATA')]; add_columns(analysisDF)
print_parameters(analysisDF,'N50EPC, (TA)3-containing')
bins = np.linspace(-10,10,20)
plt.hist(N50EPC_enr_df['log2_score'], bins, alpha=0.5 ,color = 'blue', edgecolor='black',\
         density=True, label='N50EPC, all')
plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'red', edgecolor='black',\
         density=True, label='N50EPC cont. (TA)3')
plt.legend(loc='upper left')
plt.show()


In [ ]:
## Bar plots of different versions of same motif (comparing small deviations from consensus)
## Here: the TATATA motif

def list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                               & (analysisDF['UTR'].str.contains(sequence_list[1])==False)][variable]
            #print 'properties of sequences containing motif ' + motif + ', consensus version NOT present; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1
        
def list_params_avg_motif_effects__no_exclusion(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1

        
## consensus excluded from deviant versions
print 'Plot with TATATA sequences excluded from analyses of alternative versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TATATA',\
               'AATATA','TTTATA','TAAATA','TATTTA','TATAAA','TATATT','AAATTT','TTTAAA','AAAAAA','TTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('TATATA motif (for alternative versions, \nexclude sequences also containing TATATA)')

plt.show()

print names_list
print means_list
print N_list
print

## consensus NOT excluded from deviant versions
print 'Plot with TATATA sequences NOT excluded from analyses of alternative motif versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TATATA',\
               'AATATA','TTTATA','TAAATA','TATTTA','TATAAA','TATATT','AAATTT','TTTAAA','AAAAAA','TTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__no_exclusion(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('TATATA motif (for alternative versions, \ndo NOT exclude sequences also containing TATATA)')

plt.show()

print names_list
print means_list
print N_list
print

In [ ]:
## Bar plots of different versions of same motif (comparing small deviations from consensus)
## Here: the ATATAT motif [as comparison to TATATA, see above]


def list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                               & (analysisDF['UTR'].str.contains('TATATA')==False)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                                & (analysisDF['UTR'].str.contains('TATATA')==False)\
                                & (analysisDF['UTR'].str.contains(sequence_list[1])==False)][variable]
            #print 'properties of sequences containing motif ' + motif + ', consensus version NOT present; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1
        
def list_params_avg_motif_effects__no_exclusion(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                               & (analysisDF['UTR'].str.contains('TATATA')==False)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                               & (analysisDF['UTR'].str.contains('TATATA')==False)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1

def list_params_avg_motif_effects__exclude_consensus_but_not_TA3(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                                & (analysisDF['UTR'].str.contains(sequence_list[1])==False)][variable]
            #print 'properties of sequences containing motif ' + motif + ', consensus version NOT present; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1
        
## consensus excluded from deviant versions
print 'Plot with ATATAT sequences excluded from analyses of alternative versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','ATATAT',\
               'TTATAT','AAATAT','ATTTAT','ATAAAT','ATATTT','ATATAA','AAATTT','TTTAAA','AAAAAA','TTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')


plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('ATATAT motif (exclude seqs also cont. TATATA from all);'\
          +'\nfor alt. versions, *exclude* seqs also containing ATATAT)')

plt.show()

print names_list
print means_list
print N_list
print

## consensus NOT excluded from deviant versions
print 'Plot with ATATAT sequences NOT excluded from analyses of alternative motif versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','ATATAT',\
               'TTATAT','AAATAT','ATTTAT','ATAAAT','ATATTT','ATATAA','AAATTT','TTTAAA','AAAAAA','TTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__no_exclusion(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('ATATAT motif (exclude seqs also cont. TATATA from all);'\
          +'\nfor alt. versions, do NOT exclude seqs also containing ATATAT)')

plt.show()

print names_list
print means_list
print N_list
print


In [ ]:
## Bar plots of different versions of same motif (comparing small deviations from consensus)
## Here: the TATATA motif, replacing A or T with (A-->G, T-->C)

def list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                               & (analysisDF['UTR'].str.contains(sequence_list[1])==False)][variable]
            #print 'properties of sequences containing motif ' + motif + ', consensus version NOT present; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1
        
def list_params_avg_motif_effects__no_exclusion(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1

        
## consensus excluded from deviant versions
print 'Plot with TATATA sequences excluded from analyses of alternative versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TATATA',\
               'CATATA','TGTATA','TACATA','TATGTA','TATACA','TATATG','AAATTT','TTTAAA','AAAAAA','TTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('TATATA motif and T->C, A->G subs at each site (for alt. versions, \nexclude sequences also containing TATATA)')

plt.show()

print names_list
print means_list
print N_list
print

## consensus NOT excluded from deviant versions
print 'Plot with TATATA sequences NOT excluded from analyses of alternative motif versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TATATA',\
               'CATATA','TGTATA','TACATA','TATGTA','TATACA','TATATG','AAATTT','TTTAAA','AAAAAA','TTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__no_exclusion(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('TATATA motif and T->C, A->G subs at each site (for alt. versions, \ndo NOT exclude sequences also containing TATATA)')

plt.show()

print names_list
print means_list
print N_list
print

In [ ]:
## Bar plots of different versions of same motif (comparing small deviations from consensus)
## Here: the TATATATA, aka (TA)4, motif

def list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                               & (analysisDF['UTR'].str.contains(sequence_list[1])==False)][variable]
            #print 'properties of sequences containing motif ' + motif + ', consensus version NOT present; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1
        
def list_params_avg_motif_effects__no_exclusion(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1

        
## consensus excluded from deviant versions
print 'Plot with (TA)4 sequences excluded from analyses of alternative versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TATATATA',\
               'AATATATA','TTTATATA','TAAATATA','TATTTATA','TATAAATA','TATATTTA','TATATAAA','TATATATT',\
               'AAAATTTT','TTTTAAAA','AAAAAAAA','TTTTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('(TA)4 motif (for alternative versions, \nexclude sequences also containing TATATATA)')

plt.show()

print names_list
print means_list
print N_list
print

## consensus NOT excluded from deviant versions
print 'Plot with (TA)4 sequences NOT excluded from analyses of alternative motif versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TATATATA',\
               'AATATATA','TTTATATA','TAAATATA','TATTTATA','TATAAATA','TATATTTA','TATATAAA','TATATATT',\
               'AAAATTTT','TTTTAAAA','AAAAAAAA','TTTTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__no_exclusion(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('(TA)4 motif (for alternative versions, \ndo NOT exclude sequences also containing TATATATA)')

plt.show()

print names_list
print means_list
print N_list
print

In [ ]:
## Bar plots of different versions of same motif (comparing small deviations from consensus)
## Here: the TATATATATA, aka (TA)5, motif

def list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                               & (analysisDF['UTR'].str.contains(sequence_list[1])==False)][variable]
            #print 'properties of sequences containing motif ' + motif + ', consensus version NOT present; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1
        
def list_params_avg_motif_effects__no_exclusion(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1

        
## consensus excluded from deviant versions
print 'Plot with (TA)5 sequences excluded from analyses of alternative versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TATATATATA',\
               'AATATATATA','TTTATATATA','TAAATATATA','TATTTATATA','TATAAATATA','TATATTTATA','TATATAAATA','TATATATTTA',\
               'TATATATAAA','TATATATATT','AAAAATTTTT','TTTTTAAAAA','AAAAAAAAAA','TTTTTTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue','blue','blue',\
                   'blue','blue','blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3.5)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('(TA)5 motif (for alternative versions, \nexclude sequences also containing (TA)5)')

plt.show()

print names_list
print means_list
print N_list
print

## consensus NOT excluded from deviant versions
print 'Plot with (TA)5 sequences NOT excluded from analyses of alternative motif versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TATATATATA',\
               'AATATATATA','TTTATATATA','TAAATATATA','TATTTATATA','TATAAATATA','TATATTTATA','TATATAAATA','TATATATTTA',\
               'TATATATAAA','TATATATATT','AAAAATTTTT','TTTTTAAAAA','AAAAAAAAAA','TTTTTTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__no_exclusion(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue','blue','blue',\
                   'blue','blue','blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3.5)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('(TA)5 motif (for alternative versions, \ndo NOT exclude sequences also containing (TA)5)')

plt.show()

print names_list
print means_list
print N_list
print

In [ ]:
### Figures for motif effects: absolute and relative effects of each motif on random 50-mer library enrichment & native mRNA half-life ###

In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Here: (TA)3 efficiency element

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TATATA'

minDist = len(motif)/2
print minDist

numShuffles = 10

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

print shuffleList

print

means_list = []
sems_list = []
names_list = []
N_list = []

shuffles = shuffleList[1:]  # to exclude the consensus motif itself

dataframes_list = [ ('rel_log2_score, all NOT including motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains('TATATA')==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, TA3, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains('TATATA'))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                       ('rel_log2_score, TA3_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains('TATATA')==False)\
                                            & (enrichment_df['UTR'].str.contains('|'.join(shuffles)))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]
                    


for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

        
native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all NOT including motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains('TATATA')==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), TA3',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains('TATATA')]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), TA3_shuffles',\
                        enrichment_df_native[(enrichment_df_native['UTR'].str.contains('TATATA')==False)\
                                            & (enrichment_df_native['UTR'].str.contains('|'.join(shuffles)))]\
                       [native_dataset]) ]


    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log2-enrichment)
## Here: (TA)3 efficiency element 

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TATATA' 

minDist = len(motif)/2
print minDist

numShuffles = 10

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

print shuffleList
 
print

means_list = []
sems_list = []
names_list = []
N_list = []

shuffles = shuffleList[1:]  # to exclude the consensus motif itself

dataframes_list = [ ('log2_score, all NOT including motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains('TATATA')==False)]\
                       ['log2_score']),\
                      ('log2_score, TA3, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains('TATATA'))]\
                       ['log2_score']),\
                       ('log2_score, TA3_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains('TATATA')==False)\
                                            & (enrichment_df['UTR'].str.contains('|'.join(shuffles)))]\
                       ['log2_score']) ]
                    


for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

    

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Here: T5ATA efficiency element

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TTTTTATA' 

minDist = len(motif)/2
print minDist

numShuffles = 15

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

print shuffleList

print

means_list = []
sems_list = []
names_list = []
N_list = []

shuffles = shuffleList[1:]  # to exclude the consensus motif itself



dataframes_list = [ ('rel_log2_score, all seqs WITHOUT T5ATA, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, T5ATA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, T5ATA_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]
                   


for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all seqs WITHOUT T5ATA',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), T5ATA',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), T5ATA_shuffles',\
                    enrichment_df_native[(enrichment_df_native['UTR'].str.contains(motif)==False)\
                                        & (enrichment_df_native['UTR']\
                                               .str.contains(('|'.join(shuffles))))\
                                        ]\
                   [native_dataset])\
                      ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')
    
plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## Here: T5ATA efficiency element

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TTTTTATA' 

minDist = len(motif)/2
print minDist

numShuffles = 15

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

print shuffleList

print

means_list = []
sems_list = []
names_list = []
N_list = []

shuffles = shuffleList[1:]  # to exclude the consensus motif itself



dataframes_list = [ ('log2_score, all seqs WITHOUT T5ATA, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score, T5ATA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, T5ATA_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['log2_score']) ]
                   


for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            
bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different versions of same motif (comparing small deviations from consensus)
## Here: the T5ATA motif

def list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[(analysisDF['UTR'].str.contains(motif))\
                               & (analysisDF['UTR'].str.contains(sequence_list[1])==False)][variable]
            #print 'properties of sequences containing motif ' + motif + ', consensus version NOT present; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1
        
def list_params_avg_motif_effects__no_exclusion(analysisDF,variable,sequence_list):
    i=0
    for motif in sequence_list:
        if i<2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        if i>=2:
            testDF = analysisDF[analysisDF['UTR'].str.contains(motif)][variable]
            #print 'properties of sequences containing motif ' + motif + '; mean, sem, and N:'
        means_list.append(testDF.mean())
        #print testDF.mean()
        sems_list.append(testDF.sem())
        #print testDF.sem()
        N_list.append(testDF.shape[0])
        #print testDF.shape[0]
        names_list.append(sequence_list[i])
        #print
        i=i+1

        
## consensus excluded from deviant versions
print 'Plot with T5ATA sequences excluded from analyses of alternative versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TTTTTATA',\
               'ATTTTATA','TATTTATA','TTATTATA','TTTATATA','TTTTAATA','TTTTTTTA','TTTTTAAA','TTTTTATT',\
               'AATTTTTT','TTTTTTAA','AAAAAAAA','TTTTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__exclude_consensus(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue','blue','blue',\
                   'blue','blue','blue']
edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('T5ATA motif (for alternative versions, \nexclude sequences also containing T5ATA)')

plt.show()

print names_list
print means_list
print N_list
print

## consensus NOT excluded from deviant versions
print 'Plot with T5ATA sequences NOT excluded from analyses of alternative motif versions:'
means_list = []
sems_list = []
names_list = []
N_list = []
motifs_list = ('','TTTTTATA',\
               'ATTTTATA','TATTTATA','TTATTATA','TTTATATA','TTTTAATA','TTTTTTTA','TTTTTAAA','TTTTTATT',\
               'AATTTTTT','TTTTTTAA','AAAAAAAA','TTTTTTTT')
analysisDF = results_df
variable = 'log2_score'
list_params_avg_motif_effects__no_exclusion(analysisDF,variable,motifs_list)

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue','blue',\
               'blue','blue','blue','blue','blue',\
                   'blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 3)
plt.xlabel('motifs list index')
plt.ylabel('average log2 enrichment')
plt.title('T5ATA motif (for alternative versions, \ndo NOT exclude sequences also containing T5ATA)')

plt.show()

print names_list
print means_list
print N_list
print

In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Here: consensus positioning element, AAWAAA

print

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'AATAAA|AAAAAA'
control_motif = 'AAATTT'


dataframes_list = [ ('rel_log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, AAWAAA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, AAATTT, our data',\
                        enrichment_df[((enrichment_df['UTR'].str.contains(motif))==False) &\
                                        (enrichment_df['UTR'].str.contains(control_motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), alls eqs WITHOUT motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), AAWAAA',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), AAATTT',\
                    enrichment_df_native[((enrichment_df_native['UTR'].str.contains(motif))==False) &\
                                        (enrichment_df_native['UTR'].str.contains(control_motif))]\
                   [native_dataset])\
                      ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## Here: consensus positioning element, AAWAAA

print

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'AATAAA|AAAAAA' 
control_motif = 'AAATTT'


dataframes_list = [ ('log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score, AAWAAA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, AAATTT, our data',\
                        enrichment_df[((enrichment_df['UTR'].str.contains(motif))==False) &\
                                        (enrichment_df['UTR'].str.contains(control_motif))]\
                       ['log2_score']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

        

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Here: GCGCGC control motif

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'GCGCGC' 

minDist = len(motif)/2
print minDist

numShuffles = 10

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

print shuffleList

print

means_list = []
sems_list = []
names_list = []
N_list = []

shuffles = shuffleList[1:]  # to exclude the consensus motif itself

motif = 'GCGCGC'   


dataframes_list = [ ('rel_log2_score, all seqs WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, GCGCGC, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, GCGCGC shuffles, our data',\
                        enrichment_df[((enrichment_df['UTR'].str.contains(motif))==False) &\
                                        (enrichment_df['UTR'].str.contains(('|'.join(shuffles))))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]


for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all seqs WITHOUT motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), GCGCGC',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), GCGCGC shuffles',\
                    enrichment_df_native[((enrichment_df_native['UTR'].str.contains(motif))==False) &\
                                        (enrichment_df_native['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                   [native_dataset])\
                      ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

    #print names_list
    #print means_list
    #print sems_list
    #print N_list

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)

plt.ylim(0.9, 1.3)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## Here: GCGCGC control motif

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'GCGCGC' 

minDist = len(motif)/2
print minDist

numShuffles = 10

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

print shuffleList

print

means_list = []
sems_list = []
names_list = []
N_list = []

shuffles = shuffleList[1:]  # to exclude the consensus motif itself

motif = 'GCGCGC'   


dataframes_list = [ ('log2_score, all seqs WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score, GCGCGC, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, GCGCGC shuffles, our data',\
                        enrichment_df[((enrichment_df['UTR'].str.contains(motif))==False) &\
                                        (enrichment_df['UTR'].str.contains(('|'.join(shuffles))))]\
                       ['log2_score']) ]


for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)

plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Here: Puf1/Puf2 binding site = UAAUNNNUAAU motif

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TAAT...TAAT' #'.' = regex term for any character (N) 

minDist = len(motif)/2
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TAAT...TAAT'

dataframes_list = [ ('rel_log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, TAATNNNTAAT, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, TAATNNNTAAT_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all WITHOUT motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), TAATNNNTAAT',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), TAATNNNTAAT_shuffles',\
                    enrichment_df_native[(enrichment_df_native['UTR'].str.contains(motif)==False)\
                                        & (enrichment_df_native['UTR']\
                                               .str.contains(('|'.join(shuffles))))\
                                        ]\
                   [native_dataset])\
                      ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## Here: Puf1/Puf2 binding site = UAAUNNNUAAU motif

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TAAT...TAAT' #'.' = regex term for any character (N) 

minDist = len(motif)/2
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TAAT...TAAT'

dataframes_list = [ ('log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score, TAATNNNTAAT, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, TAATNNNTAAT_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['log2_score']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
## -- RELATIVE effects on E, native gene half-life
## Here: Puf3 binding site motif = UGUANAUA 

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TGTA.ATA' 

minDist = len(motif)/2
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TGTA.ATA'

dataframes_list = [ ('rel_log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, Puf3 consensus = TGTANATA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, TGTANATA_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all seqs WITHOUT motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), Puf3 consensus = TGTANATA',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), TGTANATA_shuffles',\
                    enrichment_df_native[(enrichment_df_native['UTR'].str.contains(motif)==False)\
                                        & (enrichment_df_native['UTR']\
                                               .str.contains(('|'.join(shuffles))))\
                                        ]\
                   [native_dataset])\
                      ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print



In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## Here: Puf3 binding site motif = UGUANAUA 

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TGTA.ATA' 

minDist = len(motif)/2
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TGTA.ATA'

dataframes_list = [ ('log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score Puf3 consensus = TGTANATA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, TGTANATA_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['log2_score']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print



In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Here: Puf4 binding site motif = UGUANANUA

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TGTA.A.TA' 

minDist = len(motif)/2
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TGTA.A.TA'
print motif

dataframes_list = [ ('rel_log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, Puf4 consensus = TGTANANTA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, TGTANANTA_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all seqs WITHOUT motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), Puf4 consensus = TGTANANTA',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), TGTANANTA_shuffles',\
                    enrichment_df_native[(enrichment_df_native['UTR'].str.contains(motif)==False)\
                                        & (enrichment_df_native['UTR']\
                                               .str.contains(('|'.join(shuffles))))\
                                        ]\
                   [native_dataset])\
                      ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.6, 1.5)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print



In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## Here: Puf4 binding site motif = UGUANANUA 


## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TGTA.A.TA' 

minDist = len(motif)/2
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TGTA.A.TA'
print motif

dataframes_list = [ ('log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score, Puf4 consensus = TGTANANTA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, TGTANANTA_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['log2_score']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            
bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print



In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Puf5 binding site motif = UGUANNNNUA 

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TGTA....TA' 

minDist = len(motif)/2
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TGTA....TA'
print motif

dataframes_list = [ ('rel_log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, Puf5 consensus = TGTANNNNTA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, TGTANNNNTA_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all seqs WITHOUT motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), Puf5 consensus = TGTANNNNTA',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), TGTANNNNTA_shuffles',\
                    enrichment_df_native[(enrichment_df_native['UTR'].str.contains(motif)==False)\
                                        & (enrichment_df_native['UTR']\
                                               .str.contains(('|'.join(shuffles))))\
                                        ]\
                   [native_dataset])\
                      ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])


bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()


print names_list
print N_list
print means_list
print sems_list
print



In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## Puf5 binding site motif = UGUANNNNUA 

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TGTA....TA' 

minDist = len(motif)/2
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TGTA....TA'
print motif

dataframes_list = [ ('log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score, Puf5 consensus = TGTANNNNTA, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, TGTANNNNTA_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['log2_score']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            
bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print



In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Here: Puf6 binding site motif = UUGU 

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TTGT' 

minDist = len(motif)/2
print minDist

numShuffles = 3

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TTGT'
print motif

dataframes_list = [ ('rel_log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, Puf6 site = TTGT, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, TTGT_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all seqs WITHOUT motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), Puf6 site = TTGT',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), TTGT_shuffles',\
                    enrichment_df_native[(enrichment_df_native['UTR'].str.contains(motif)==False)\
                                        & (enrichment_df_native['UTR']\
                                               .str.contains(('|'.join(shuffles))))\
                                        ]\
                   [native_dataset])\
                      ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()


print names_list
print N_list
print means_list
print sems_list
print



In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## Here: Puf6 binding site motif = UUGU 

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TTGT' 

minDist = len(motif)/2
print minDist

numShuffles = 3

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList



#print
shuffles = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print shuffles

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TTGT'
print motif

dataframes_list = [ ('log2_score, all WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score, Puf6 site = TTGT, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, TTGT_shuffles, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['log2_score']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])


bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print



In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log-2 enrichment)
## polyU_8X = UUUUUUUU;   first 25 nt, last 25 nt, ..., A4T4 shuffles control (5 bars total, see below)

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TTTTAAAA' # shuffle this to generate %AU control (4U's, 4 A's)

minDist = len(motif)/2  ## this is in addition to extra filter to remove sequences with 'TATA' content (see below)
print minDist

numShuffles = 35

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList) & (('TATA' in shuffleX) == False)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList

#set and print
shuffles = shuffleList#[1:]  ## [1:] to exclude the consensus motif itself
#print motif
print shuffles

print

means_list = []
sems_list = []
names_list = []
N_list = []

motif = 'TTTTTTTT'


dataframes_list = [ ('log2_score, all seqs WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['log2_score']),\
                      ('log2_score, polyU_8X, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, polyU_8X in first 25 nt, our data',\
                       enrichment_df[(enrichment_df['UTR'].str[1:25].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, polyU_8X in last 25 nt, our data',\
                       enrichment_df[(enrichment_df['UTR'].str[26:50].str.contains(motif))]\
                       ['log2_score']),\
                   ('log2_score, T4A4 control motif, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['log2_score']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- RELATIVE EFFECTS ON E, half-life
## Here: polyU_8X = UUUUUUUU, with shuffles of T4A4 (inc T4A4) as control

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = 'TTTTAAAA' # T4A4 shuffles control generation (for 4 U's + 4 A's control)

minDist = len(motif)/2  ## this is in addition to extra filter to remove sequences with 'TATA' content (see below)
print minDist

numShuffles = 35

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList) & (('TATA' in shuffleX) == False)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList

#set and print
shuffles = shuffleList#[1:]  ## [1:] to exclude the consensus motif itself
#print motif
print shuffles

print

means_list = []
sems_list = []
names_list = []
N_list = []


motif = 'TTTTTTTT'
controlMotif = 'TTAAATTT'
print motif
print controlMotif


dataframes_list = [ ('rel_log2_score, all seqs WITHOUT motif, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                      ('rel_log2_score, polyU_8X, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif))]\
                       ['rel_log2_score(rescaled+mean-norm)']),\
                   ('rel_log2_score, T4A4 shuffles control, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       ['rel_log2_score(rescaled+mean-norm)']) ]



for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

native_dataset_list = ['rel_Geisberg/min(mean-norm)']
for native_dataset in native_dataset_list:

    print native_dataset
    
    dataframes_list = [ ('rel_nativeUTRs_halflife(/min), all seqs WITHOUT motif',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)==False]\
                       [native_dataset]),\
                      ('rel_nativeUTRs_halflife(/min), polyU_8X',\
                       enrichment_df_native[enrichment_df_native['UTR'].str.contains(motif)]\
                       [native_dataset]),\
                       ('rel_nativeUTRs_halflife(/min), T4A4 shuffles control',\
                       enrichment_df_native[(enrichment_df_native['UTR'].str.contains(motif)==False)\
                                            & (enrichment_df_native['UTR']\
                                               .str.contains(('|'.join(shuffles))))]\
                       [native_dataset]) ]
    

    for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue','green','green','green']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0.9, 1.3)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## Bar plots of different motifs, different datasets, for N50C Enr (E) data as well as native UTR half-life data;
## plotting effects for one motif at a time (& the associated controls).
#  -- ABSOLUTE EFFECTS ON E (log2 enr)
## Here: polyU (8x) vs U-rich controls with no more than two sequential U nucleotides 

## make shuffles (using my existing shufflemaker script, reproduced here)

motif = '....TTTTTTTT....' # NNNNUUUUUUUUNNNN polyU vs U-rich control (for 8 U's + 8 N's)

minDist = len(motif)/2  ## this is in addition to new filter removing sequences with 'TTT' content (see below)
print minDist

numShuffles = 50

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)\
            & (('TTT' in shuffleX) == False)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

#print shuffleList

#set and print: motif 1
motif_1 = motif
shuffles_1 = shuffleList[1:]  ## [1:] to exclude the consensus motif itself
print motif_1
print shuffles_1


means_list = []
sems_list = []
names_list = []
N_list = []

dataframes_list = [ ('log2_score, seqs WITHOUT N4U8N4, our data',\
                         enrichment_df[(enrichment_df['UTR'].str.contains(motif_1)==False)]\
                       ['log2_score']),\
                      ('log2_score, seqs WITH N4U8N4, our data',\
                       enrichment_df[(enrichment_df['UTR'].str.contains(motif_1))]\
                       ['log2_score']),\
                   ('log2_score, seqs WITH SHUFFLES of N4U8N4 only, our data',\
                        enrichment_df[(enrichment_df['UTR'].str.contains(motif_1)==False)\
                                           & (enrichment_df['UTR'].str.contains('TATATA')==False)\
                                      & (enrichment_df['UTR']\
                                               .str.contains(('|'.join(shuffles_1))))]\
                       ['log2_score']) ]
print


for dataframe in dataframes_list:
        means_list.append(dataframe[1].mean())
        sems_list.append(dataframe[1].sem())
        names_list.append(dataframe[0])
        N_list.append(dataframe[1].shape[0])

            

bar_x = list(range(len(names_list)))
#print bar_x
colors_list = ['blue','blue','blue']

edge_colors_list = []
for member in bar_x:
    edge_colors_list.append('black')

plt.bar(x=bar_x, height=means_list, yerr=sems_list, capsize=4.0, color=colors_list,\
        edgecolor=edge_colors_list)#, tick_label=names_list)
plt.ylim(0, 2.75)

plt.show()

print names_list
print N_list
print means_list
print sems_list
print


In [ ]:
## further analyses of (TA)3-containing sequences

In [ ]:
N50C_df['GC%'] = N50C_df['UTR'].apply(GC)
N50C_df['AT%'] = 100 - N50C_df['GC%']

In [ ]:
import re

In [ ]:
N50C_df['(TA)3_pos(first)'] = N50C_df['UTR'].str.find('TATATA')
#N50C_df['AAWAAA_pos(first)'] = N50C_df['UTR'].str.find('AATAAA|AAAAAA')

def regex_search_start(x):
    try:
        return re.search('AATAAA|AAAAAA', str(x)).start()
    except:
        return np.nan

N50C_df['AAWAAA_pos(first)'] = N50C_df['UTR'].apply\
(lambda x: regex_search_start(x))
#(lambda x: (re.search('AATAAA|AAAAAA', str(x))))

N50C_df.loc[N50C_df['(TA)3_pos(first)']==-1,'(TA)3_pos(first)']\
= np.nan

N50C_df.loc[N50C_df['AAWAAA_pos(first)']==-1,'AAWAAA_pos(first)']\
= np.nan

N50C_df.loc[N50C_df['(TA)3_pos(first)']>=0,'(TA)3_pos(first)']\
= N50C_df['(TA)3_pos(first)'] + 1

N50C_df.loc[N50C_df['AAWAAA_pos(first)']>=0,'AAWAAA_pos(first)']\
= N50C_df['AAWAAA_pos(first)'] + 1

In [ ]:
print N50C_df.shape[0]
N50C_df_w_TA3 = N50C_df[N50C_df['UTR'].str.contains('TATATA')]
print N50C_df_w_TA3.shape[0]

In [ ]:
# analysis of N50C sequences with multiple occurences of the consensus efficiency element, UAUAUA

In [ ]:
N50C_df['(TA)3_num_instances'] = N50C_df['UTR'].str.count('TATATA')
N50C_df_w_TA3 = N50C_df[N50C_df['UTR'].str.contains('TATATA')]
N50C_df_w_mult_TA3_nonoverlapping = N50C_df[N50C_df['(TA)3_num_instances']>=2]

print 'N50C_all'; print N50C_df.shape[0]; print np.mean(N50C_df['log2_score']); print sem(N50C_df['log2_score']); print
print 'N50C_containing_TA3'; print N50C_df_w_TA3.shape[0]; print np.mean(N50C_df_w_TA3['log2_score']);\
      print sem(N50C_df_w_TA3['log2_score']); print
print 'N50C_containing_2X_TA3(nonoverlapping)'; print N50C_df_w_mult_TA3_nonoverlapping.shape[0];\
         print np.mean(N50C_df_w_mult_TA3_nonoverlapping['log2_score']);\
        print sem(N50C_df_w_mult_TA3_nonoverlapping['log2_score']); print

#print 

def make_shuffles(motif_input,numShuffles_input):
    
    motif = motif_input
    numShuffles = numShuffles_input
    
    minDist = len(motif)/2
    #print minDist

    shuffleList = []
    shuffleList.append(motif)
    #print shuffleList

    for i in range (0,numShuffles): #(0,5):
        continueLoop = True
        while(continueLoop == True):
            shuffleX = motif
            #print shuffleX
            shuffleX_list = list(shuffleX)
            random.shuffle(shuffleX_list)
            shuffleX = ''.join(shuffleX_list)
            #print shuffleX
            dist = lev.hamming(shuffleX, motif)
            if ((dist >= minDist) & (shuffleX not in shuffleList)) :
                #print "success"
                shuffleList.append(shuffleX)
                continueLoop = False
            #else:
                #print "fail, try again"

    #print;print; print shuffleList
    return shuffleList

motif='TATATA'
numShuffles = 10
shuffleList = make_shuffles(motif,numShuffles)
shuffles = shuffleList[1:]

#print
N50C_df_w_1X_TA3 = N50C_df[N50C_df['(TA)3_num_instances']==1]

print 'N50C_containing_1X_TA3'; print N50C_df_w_1X_TA3.shape[0]; print np.mean(N50C_df_w_1X_TA3['log2_score']);\
      print sem(N50C_df_w_1X_TA3['log2_score']); print





In [ ]:
# bin TA3-cont. sequences by enrichment

In [ ]:
N50C_df_w_TA3_bin1 = N50C_df_w_TA3[N50C_df_w_TA3['log2_score']<-3]
print N50C_df_w_TA3_bin1.shape[0]

N50C_df_w_TA3_bin2 = N50C_df_w_TA3[(N50C_df_w_TA3['log2_score']>=-3)\
                                   &(N50C_df_w_TA3['log2_score']<-1)]
print N50C_df_w_TA3_bin2.shape[0]

N50C_df_w_TA3_bin3 = N50C_df_w_TA3[(N50C_df_w_TA3['log2_score']>=-1)\
                                   &(N50C_df_w_TA3['log2_score']<1)]
print N50C_df_w_TA3_bin3.shape[0]

N50C_df_w_TA3_bin4 = N50C_df_w_TA3[(N50C_df_w_TA3['log2_score']>=1)\
                                   &(N50C_df_w_TA3['log2_score']<3)]
print N50C_df_w_TA3_bin4.shape[0]

N50C_df_w_TA3_bin5 = N50C_df_w_TA3[(N50C_df_w_TA3['log2_score']>=3)\
                                   &(N50C_df_w_TA3['log2_score']<6)]
print N50C_df_w_TA3_bin5.shape[0]

N50C_df_w_TA3_bin6 = N50C_df_w_TA3[(N50C_df_w_TA3['log2_score']>=6)\
                                   &(N50C_df_w_TA3['log2_score']<9)]
print N50C_df_w_TA3_bin6.shape[0]

In [ ]:
N50C_df_bin1 = N50C_df[N50C_df['log2_score']<-3]
print N50C_df_bin1.shape[0]

N50C_df_bin2 = N50C_df[(N50C_df['log2_score']>=-3)\
                                   &(N50C_df['log2_score']<-1)]
print N50C_df_bin2.shape[0]

N50C_df_bin3 = N50C_df[(N50C_df['log2_score']>=-1)\
                                   &(N50C_df['log2_score']<1)]
print N50C_df_bin3.shape[0]

N50C_df_bin4 = N50C_df[(N50C_df['log2_score']>=1)\
                                   &(N50C_df['log2_score']<3)]
print N50C_df_bin4.shape[0]

N50C_df_bin5 = N50C_df[(N50C_df['log2_score']>=3)\
                                   &(N50C_df['log2_score']<6)]
print N50C_df_bin5.shape[0]

N50C_df_bin6 = N50C_df[(N50C_df['log2_score']>=6)\
                                   &(N50C_df['log2_score']<9)]
print N50C_df_bin6.shape[0]

In [ ]:
N50C_df_w_TA3_and_AAWAAA = N50C_df_w_TA3[N50C_df_w_TA3['UTR'].str.contains('AATAAA|AAAAAA')]
N50C_df_w_TA3_and_AAWAAA.head(3)
pos_before_eff_df = N50C_df_w_TA3_and_AAWAAA[N50C_df_w_TA3_and_AAWAAA['(TA)3_pos(first)']\
                                      > N50C_df_w_TA3_and_AAWAAA['AAWAAA_pos(first)']]
eff_before_pos_df = N50C_df_w_TA3_and_AAWAAA[N50C_df_w_TA3_and_AAWAAA['(TA)3_pos(first)']\
                                      < N50C_df_w_TA3_and_AAWAAA['AAWAAA_pos(first)']]
print N50C_df_w_TA3_and_AAWAAA.shape[0]
print pos_before_eff_df.shape[0]
print eff_before_pos_df.shape[0]

In [ ]:
N50C_df.name = 'N50C_df'
N50C_df_w_TA3.name = 'N50C_df_w_TA3'

N50C_df_w_TA3_and_AAWAAA = N50C_df_w_TA3[N50C_df_w_TA3['UTR'].str.contains('AATAAA|AAAAAA')]
N50C_df_w_TA3_and_AAWAAA.name = 'N50C_df_w_TA3_and_AAWAAA'

pos_before_eff_df = N50C_df_w_TA3_and_AAWAAA[N50C_df_w_TA3_and_AAWAAA['(TA)3_pos(first)']\
                                      > N50C_df_w_TA3_and_AAWAAA['AAWAAA_pos(first)']]
pos_before_eff_df.name = 'pos_before_eff_df'

eff_before_pos_df = N50C_df_w_TA3_and_AAWAAA[N50C_df_w_TA3_and_AAWAAA['(TA)3_pos(first)']\
                                      < N50C_df_w_TA3_and_AAWAAA['AAWAAA_pos(first)']]
eff_before_pos_df.name = 'eff_before_pos_df'

print N50C_df.name + " mean log2 score = " + str((N50C_df['log2_score']).mean())
print N50C_df_w_TA3.name + " mean log2 score = " + str((N50C_df_w_TA3['log2_score']).mean())
print N50C_df_w_TA3_and_AAWAAA.name + " mean log2 score = " + str((N50C_df_w_TA3_and_AAWAAA['log2_score']).mean())
print "N50C with pos --> eff mean log2 score = " + str((pos_before_eff_df['log2_score']).mean())
print "N50C with eff --> pos mean log2 score = " + str((eff_before_pos_df['log2_score']).mean())
print
print


dfs_list = [N50C_df_w_TA3_bin2, N50C_df_w_TA3_bin3, N50C_df_w_TA3_bin4,\
           N50C_df_w_TA3_bin5, N50C_df_w_TA3_bin6]   #exclude bin1 because N=2 for this bin

N50C_df_w_TA3_bin2.name = 'N50C_df_w_TA3_bin2'
N50C_df_w_TA3_bin3.name = 'N50C_df_w_TA3_bin3'
N50C_df_w_TA3_bin4.name = 'N50C_df_w_TA3_bin4'
N50C_df_w_TA3_bin5.name = 'N50C_df_w_TA3_bin5'
N50C_df_w_TA3_bin6.name = 'N50C_df_w_TA3_bin6'

bins_fraction_posBeforeEff_list = []
bins_fraction_effBeforePos_list = []
bins_count_posBeforeEff_list = []
bins_count_effBeforePos_list = []
bins_count_pos_and_eff_list = []
bins_GC_mean_list = []
bins_GC_sem_list = []
bins_count_in_bin_list = []
bins_AT_mean_list = []
bins_AT_sem_list = []

for analysisDF in dfs_list:
    pos_and_eff_df = analysisDF[analysisDF['UTR'].str.contains('AAAAAA|AATAAA')]
    pos_before_eff_df = pos_and_eff_df[pos_and_eff_df['(TA)3_pos(first)']\
                                      > pos_and_eff_df['AAWAAA_pos(first)']]
    eff_before_pos_df = pos_and_eff_df[pos_and_eff_df['(TA)3_pos(first)']\
                                      < pos_and_eff_df['AAWAAA_pos(first)']]
    number_pos_and_eff = float(pos_and_eff_df.shape[0])
    number_pos_first = float(pos_before_eff_df.shape[0])
    number_eff_first = float(eff_before_pos_df.shape[0])
    fraction_pos_first = number_pos_first / number_pos_and_eff
    fraction_eff_first = number_eff_first / number_pos_and_eff
    print "For the data subset " + str(analysisDF.name) + ":"
    print "whole-bin log2 score min = " + str(analysisDF['log2_score'].min())
    print "whole-bin log2 score max = " + str(analysisDF['log2_score'].max())
    print "number UTRs containing both pos and eff elements:  " + str(number_pos_and_eff)
    print "number UTRs containing both, pos BEFORE eff (pos --> eff):  " + str(number_pos_first)
    print "number UTRs containing both, pos AFTER eff (eff --> pos):  " + str(number_eff_first)
    print "fraction with pos elem --> eff elem: "
    print fraction_pos_first
    print "fraction with eff elem --> pos elem:"
    print fraction_eff_first
    print "mean log2 score of entire bin = " + str(analysisDF['log2_score'].mean())
    print "mean log2 score of bin sequences containing pos AND eff elems = " + str(pos_and_eff_df['log2_score'].mean())
    print "mean log2 score of bin sequences containing pos --> eff = " + str(pos_before_eff_df['log2_score'].mean())
    print "mean log2 score of bin sequences containing eff --> pos = " + str(eff_before_pos_df['log2_score'].mean())
    print
    bins_fraction_posBeforeEff_list.append(fraction_pos_first)
    bins_fraction_effBeforePos_list.append(fraction_eff_first)
    bins_count_posBeforeEff_list.append(number_pos_first)
    bins_count_effBeforePos_list.append(number_eff_first)
    bins_count_pos_and_eff_list.append(number_pos_and_eff)
    bins_GC_mean_list.append(analysisDF['GC%'].mean())
    bins_GC_sem_list.append(analysisDF['GC%'].sem())
    bins_count_in_bin_list.append(analysisDF.shape[0])
    bins_AT_mean_list.append(analysisDF['AT%'].mean())
    bins_AT_sem_list.append(analysisDF['AT%'].sem())
    
bar_x = ['[-3,-1)', '[-1,1)', '[1,3)', '[3,6)', '[6,9)']

plt.bar(x=bar_x, height=bins_fraction_effBeforePos_list, color='blue',\
        edgecolor='black')#, tick_label=bins_count_effBeforePos_list)
for i in range(len(bar_x)):
    plt.annotate(str(bins_count_pos_and_eff_list[i]),xy=(bar_x[i],bins_fraction_effBeforePos_list[i]+0.01))

plt.show()

bar_x = ['[-3,-1)', '[-1,1)', '[1,3)', '[3,6)', '[6,9)']

plt.bar(x=bar_x, height=bins_fraction_effBeforePos_list, color='blue',\
        edgecolor='black')#, tick_label=bins_count_effBeforePos_list)

plt.show()



In [ ]:
### positional analyses of motif effects

In [ ]:
### "Shuffle-maker" script: generate desired number of shuffled sequences derived from motif of interest. 
### The shuffles (and nonshuffled original motif) generated here are exported for downstream use
### in finding all instances of the motif and its shuffled versions in the N50 sequences using FIMO (see below)

### The example shown here is for U5AUA (alternative efficiency element)

motif = 'TTTTTATA'    #N=15  #Efficiency element v2

#'TATATA'      #N=10  #Consensus efficiency element
#'TTTTTATA'    #N=15  #Efficiency element v2
#'GCGCGC'      #N=10  #GC-rich control motif
#'UAAUNNNUAAU' #N=50  #PUF1/PUF2 motif
#'TGTANATA'    #N=50 #TGTANATA-Puf3-FINAL_CORRECT for MS
#'TGTANANTA'   #N=50 #TGTANANTA-Puf4-FINAL_CORRECT for MS
#'TGTANNNNTA'  #N=50 #TGTANNNNTA-Puf5-FINAL-CORRECT for MS
#'TTGT'        #N=3  #TTGT-Puf6-FINAL_CORRECT

minDist = len(motif)/2
print minDist

numShuffles = 15

shuffleList = []
shuffleList.append(motif)
#print shuffleList

for i in range (0,numShuffles): #(0,5):
    while(True):
        shuffleX = motif
        #print shuffleX
        shuffleX_list = list(shuffleX)
        random.shuffle(shuffleX_list)
        shuffleX = ''.join(shuffleX_list)
        #print shuffleX
        dist = lev.hamming(shuffleX, motif)
        if ((dist >= minDist) & (shuffleX not in shuffleList)) :
            #print "success"
            shuffleList.append(shuffleX)
            break
        #else:
            #print "fail, try again"

print shuffleList


In [ ]:
df_motif_shuffles_test = pandas.DataFrame({'motif':shuffleList})

In [ ]:
## Make text file of starting motif & its shuffles

length = len(shuffleList)
counter = 0
path = "[[YOUR_PATH_HERE]]/"\
+ "efficiency_TTTTTATA_with_15_shuffles" + ".txt"

#+ "efficiency_TATATA_with_10_shuffles" + ".txt" #consensus efficiency element
#+ "efficiency_TTTTTATA_with_15_shuffles" + ".txt" #alternative efficiency element
#+ "control_GCGCGC_with_10_shuffles" + ".txt" #(GC)3 control motif
#+ "PUF1&2_UAAUNNNUAAU_with_50_shuffles" + ".txt" #Puf1/Puf2 binding motif
#+ "PUF3_TGTANATA_with_50_shuffles" + ".txt"  #Puf3 binding site
#+ "Puf4_TGTANANTA_with_50_shuffles.txt"  #Puf4 binding site
#+ "Puf5_TGTANNNNTA_with_50_shuffles.txt"  #Puf5 binding site
#+ "Puf6_TTGT_with_3_shuffles.txt"  #Puf6 binding site


df_motif_shuffles_txt = open(path,"w+")

for i in range(0,length):
    df_motif_shuffles_txt.write(shuffleList[counter] + "\n" + "\n")
    counter+=1
    
df_motif_shuffles_txt.close()

In [ ]:
## Make a set of ".sites" format files for the starting motif and each of its shuffles;
## these files are used as input for running the MEME Suite program FIMO in the command line, see below

length = len(shuffleList)
counter = 0
motifName = 'efficiency_v2_TTTTTATA'      #alternative efficiency element
## ** Also need to pre-generate a folder corresponding to the name of the motif in directory 'direc', see below! **

#'efficiency_consensus_TATATA'  #consensus efficiency element
#'efficiency_v2_TTTTTATA'       #alternative efficiency element
#'control_GCGCGC'               #(GC)3 control motif'
#'PUF1&2_UAAUNNNUAAU'           #Puf1/Puf2 binding site motif 
#'Puf3_TGTANATA'                #Puf3 binding site motif
#'Puf4_TGTANANTA'               #Puf4 binding site motif
#'Puf5_TGTANNNNTA'              #Puf5 binding site motif
#'Puf6_TTGT'                    #Puf6 binding site motif


motifID = motifName + ".sites"
direc = "[[YOUR_PATH_HERE]]/" + motifName + "/"  ## remember to pre-generate the "motifName" folder

for i in range(0,length):
    if counter==0:
        path = direc + motifID
    else:
        motifID = motifName + "_shuffle" + str(counter) + ".sites" 
        path = direc + motifID
        
    df_motif_shuffles_sites = open(path,"w+")

    df_motif_shuffles_sites.write(shuffleList[counter])
    
    counter+=1
    
    df_motif_shuffles_sites.close()

In [ ]:
### Script for running sites2meme and FIMO from the MEME Suite software package
### in order to find *all perfect matches* to the motif and each of its shuffled sequences
### in the set of random 50-mer sequences interrogated by our high-throughput assay.

## NOTE that this is script is run in the command line using a local installation of the MEME Suite programs;
## will not run in iPython / Jupyter. For this reason the script is provided here as a triple-quoted literal
## that can be copied to run it in the command line.

# Example provided for the case of U5AUA motif (alt. efficiency element):

'''

#Running sites2meme to convert ".sites" files to MEME format for FIMO analysis:
sites2meme -ext .sites [[YOUR_PATH_HERE]]/efficiency_v2_TTTTTATA/ -> [[SITES2MEME_OUTPUT_DIR]]/efficiency_v2_TTTTTATA_w_shuffles_meme.txt &

#Running FIMO to find all perfect matches to motif sequence and shuffled sequences:
fimo --norc --thresh 1.6e-5 --bfile --uniform-- --oc [[FIMO_OUTPUT_DIR]]/efficiency_v2_TTTTTATA_w_shuffles_FIMOoutput [[SITES2MEME_OUTPUT_DIR]]/efficiency_v2_TTTTTATA_w_shuffles_meme.txt [[DATA_DIRECTORY]]/N50C_data.fasta &

## notes: 
## --norc --> no reverse complement, search given strand only
## --thresh --> set minimum p-value for matching. If uniform base probabilities, P-value of <1.5e-5 required for 8-residue motif ((0.25)^8)
## --o --> Create a folder called dir and write output files in it. This option is not compatible with --oc as only one output folder is allowed.
## --oc --> Create a folder called dir but if it already exists allow overwriting the contents. This option is not compatible with --o as only one output folder is allowed.
## --bfile --> Specify the source of a 0-order background model for converting a frequency matrix to a log-odds score matrix and for use in estimating the p-values of match scores. The background model normalizes for biased distribution of individual letters in the sequences. The value of file is either the path to a file in Markov Background Model Format, or one of the keywords --motif--, motif-file or --uniform--. The first two keywords cause the 0-order letter frequencies contained in the motif file to be used, and --uniform-- causes uniform letter frequencies to be used. If the background model in file is higher than 0-order, only the 0-order portion is used. If both strands are being scored, the background model is modified by averaging the frequencies of letters and their reverse complements.

'''

In [ ]:
FIMO_results_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min_df_path =\
"[[FIMO_OUTPUT_PATH]]/"\
+ "efficiency_v2_TTTTTATA_w_shuffles_FIMOoutput/"\
+ "fimo" + ".tsv"

FIMO_results_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min =\
pandas.read_csv(FIMO_results_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min_df_path,sep=None)

FIMO_results_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min.rename(columns={'sequence_name': 'seq_ID'}, inplace=True)

FIMO_results_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min.sort_values(by='motif_id').head(7)

In [ ]:
enrichment_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min = pandas.merge\
(FIMO_results_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min, enrichment_df, on='seq_ID')
enrichment_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min\
[enrichment_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min['motif_id']=='efficiency_v2_TTTTTATA']\
.sort_values(by='start').head(2)

In [ ]:
# Script for the generation of positional mean enrichment plots for motif and average of shuffled motif control sequences
# example below for alternative efficiency element, UUUUUAUA (U5AUA):

motifName = 'efficiency_v2_TTTTTATA' #15 shuffles

dataframe = enrichment_T5ATA_efficiency_v2_w_shuffles_N50C_noPC_5min

counter = 0      #initialize counter (see below)
lastMotif = 15   # lastMotif = -1 
# use lastMotif = -1 to cancel shuffle plot calculations and plotting, only plot motif positional effects (e.g. for AAWAAA)
# otherwise, lastMotif should match the # of shuffles for the motif being analyzed (e.g., 15 for U5AUA)
end = lastMotif+1

set_yLim_bool = False  # set y-limits of plots if desired (default: False)
#yMin = -0.4 ; yMax = 3.8 # example fixed y-limits (uncomment if desired)

libAndData = '_enrichment_N50C_input>=5_noPcounts'

variable = 'log2_score'

ylabel = 'mean enrichment score'

reference_dataframe = N50C_df #aka enrichment_df      
### for plotting average value across whole dataset (red horizontal line)

specialNote = ''   #(none, = default)

motif = motifName
startSite=1

enrich_vs_startPos_motif = []
SEM_enrich_vs_startPos_motif = []
startPos = []
count_at_pos = []

for startSite in range(1,51):
    startPos.append(startSite)
    mean_score = dataframe[(dataframe['motif_id']==motif)\
                                    &(dataframe['start']==startSite)][variable].mean()
    SEM_score = dataframe[(dataframe['motif_id']==motif)\
                                &(dataframe['start']==startSite)][variable].sem()
    count = dataframe[(dataframe['motif_id']==motif)\
                                &(dataframe['start']==startSite)][variable].shape[0]

    SEM_enrich_vs_startPos_motif.append(SEM_score)

    enrich_vs_startPos_motif.append(mean_score)

    count_at_pos.append(count)


plt.errorbar(startPos, enrich_vs_startPos_motif, yerr=SEM_enrich_vs_startPos_motif, ecolor='grey',\
             fmt='b.-', capsize=2, capthick=1.5) 


title = 'Motif ' + motif + ' ' + specialNote + ', N50C'  

plt.title(title)
plt.xlabel('start position')
plt.ylabel(ylabel)
if(set_yLim_bool==True):
    plt.ylim(yMin,yMax)
plt.axhline(reference_dataframe[variable].mean(),color='red')


figName = 'motif_'+motifName+' '+specialNote+libAndData

plt.show()


print count_at_pos
print np.mean(count_at_pos)


counter = 1
enrich_vs_startPos_SUM = []
SEM_enrich_vs_startPos_SUMofSQUARES = []

for i in range(1,end):
    
    motif = motifName + '_shuffle' + str(counter) 
    startSite = 1
    
    enrich_vs_startPos = []
    SEM_enrich_vs_startPos = []
   
    for startSite in range(1,51):
        #startPos.append(startSite)
        mean_score = dataframe[(dataframe['motif_id']==motif)\
                                        &(dataframe['start']==startSite)][variable].mean()
        SEM_score = dataframe[(dataframe['motif_id']==motif)\
                                    &(dataframe['start']==startSite)][variable].sem()
        count = dataframe[(dataframe['motif_id']==motif)\
                                    &(dataframe['start']==startSite)][variable].shape[0]

        SEM_enrich_vs_startPos.append(SEM_score)
        
        enrich_vs_startPos.append(mean_score)
        
        #count_at_pos.append(count)
        
    if i == 1:
        enrich_vs_startPos_SUM.append(enrich_vs_startPos)
        SEM_enrich_vs_startPos_Squared = np.square(SEM_enrich_vs_startPos)
        SEM_enrich_vs_startPos_SUMofSQUARES.append(SEM_enrich_vs_startPos_Squared)
    else:
        enrich_vs_startPos_SUM = np.add(enrich_vs_startPos_SUM, enrich_vs_startPos)
        SEM_enrich_vs_startPos_Squared = np.square(SEM_enrich_vs_startPos)
        SEM_enrich_vs_startPos_SUMofSQUARES = np.add(SEM_enrich_vs_startPos_SUMofSQUARES,\
                                                     SEM_enrich_vs_startPos_Squared)

    #print enrich_vs_startPos    
    counter+=1

if lastMotif > -1:
    #print enrich_vs_startPos_SUM
    #print str(float(lastMotif))
    enrich_vs_startPos_shuffleAVG = enrich_vs_startPos_SUM/float(lastMotif)
    enrich_vs_startPos_shuffleAVG = enrich_vs_startPos_shuffleAVG.tolist()
    enrich_vs_startPos_shuffleAVG = enrich_vs_startPos_shuffleAVG[0]
    #print enrich_vs_startPos_shuffleAVG
    SEM_enrich_vs_startPos_shuffleAVG = (1.0/float(lastMotif)) * np.sqrt(SEM_enrich_vs_startPos_SUMofSQUARES)
    SEM_enrich_vs_startPos_shuffleAVG = SEM_enrich_vs_startPos_shuffleAVG.tolist()
    SEM_enrich_vs_startPos_shuffleAVG = SEM_enrich_vs_startPos_shuffleAVG[0]
    #print SEM_enrich_vs_startPos_shuffleAVG


    plt.errorbar(startPos, enrich_vs_startPos_shuffleAVG, yerr=SEM_enrich_vs_startPos_shuffleAVG, ecolor='grey',\
                 fmt='g.-', capsize=2, capthick=1.5) #new! in progress

   
    title = 'Motif ' + motifName + ' ' + specialNote + '_avg(shuffles), N50C' 


    plt.title(title)
    plt.xlabel('start position')
    plt.ylabel(ylabel)
    if(set_yLim_bool==True):
        plt.ylim(yMin,yMax)
    plt.axhline(reference_dataframe[variable].mean(),color='red')

    
    figName = 'motif_'+motifName+' '+specialNote+'_avg(shuffles)'+libAndData 
    
    plt.show()
    

    plt.errorbar(startPos, enrich_vs_startPos_motif, yerr=SEM_enrich_vs_startPos_motif, ecolor='grey',\
                 fmt='b.-', capsize=2, capthick=1.5)
    
    plt.errorbar(startPos, enrich_vs_startPos_shuffleAVG, yerr=SEM_enrich_vs_startPos_shuffleAVG, ecolor='grey',\
                 fmt='g.-', capsize=2, capthick=1.5)

    title = 'Motif ' + motifName + ' ' + specialNote + '_VS_avg(shuffled), N50C'

    plt.title(title)
    plt.xlabel('start position')
    plt.ylabel(ylabel)
    if(set_yLim_bool==True):
        plt.ylim(yMin,yMax)
    plt.axhline(reference_dataframe[variable].mean(),color='red')

    
    figName = 'motif_'+motifName+' '+specialNote+'_VS_avg(shuffled)'+libAndData 

    plt.show()


In [ ]:
### Comprehensive analysis of average effects of k-mer sequence motifs (including for 6-mers) ###

## select dataset you want to perform the k-mer analysis on:
#v1: comprehensive 6-mer results for growth selection enrichment, N50C lib, 5 input reads min, no pseudocounts
analysisDF = N50C_df

##v2: comprehensive 6-mer results for growth selection enrichment, N50EPC lib, 5 input reads min, no pseudocounts
# analysisDF = N50EPC_enr_df


## first, make kmers list for desired kmer length:

alphabet = ['A','C','G','T']
kmer_len = 4  ### #4 #5 #6 #7 #8
kmers_list = [''.join(i) for i in itertools.product(alphabet, repeat=kmer_len)]

def sem(df):
    return np.std(df) / np.sqrt(df.shape[0])

#print kmers_list

mean_kmer_effect_list = []
sem_kmer_effect_list = []
N_kmer_effect_list = []
mean_NOkmer_effect_list = []
sem_NOkmer_effect_list = []
N_NOkmer_effect_list = []

## next, calculate average effect of kmer of each length in the desired dataset:

for kmer in kmers_list:
    #print kmer
    kmerDF = analysisDF[analysisDF['UTR'].str.contains(kmer)]
    NOkmerDF = analysisDF[analysisDF['UTR'].str.contains(kmer)==False]
    #print np.mean(kmerDF['log2_score'])
    #print np.mean(NOkmerDF['log2_score'])
    #print
    mean_kmer_effect_list.append(np.mean(kmerDF['log2_score']))
    sem_kmer_effect_list.append(sem(kmerDF['log2_score']))
    N_kmer_effect_list.append(kmerDF['log2_score'].shape[0])
    #
    mean_NOkmer_effect_list.append(np.mean(NOkmerDF['log2_score']))
    sem_NOkmer_effect_list.append(sem(NOkmerDF['log2_score']))
    N_NOkmer_effect_list.append(NOkmerDF['log2_score'].shape[0])
    
kmer_results_df = pandas.DataFrame()
kmer_results_df['kmer_sequence'] = kmers_list
kmer_results_df['kmer_enr_mean'] = mean_kmer_effect_list
kmer_results_df['kmer_enr_sem'] =  sem_kmer_effect_list
kmer_results_df['kmer_enr_N'] =  N_kmer_effect_list
kmer_results_df['NOkmer_enr_mean'] = mean_NOkmer_effect_list
kmer_results_df['NOkmer_enr_sem'] =  sem_NOkmer_effect_list
kmer_results_df['NOkmer_enr_N'] =  N_NOkmer_effect_list

kmer_results_df.head(10)

In [ ]:
kmer_results_df['kmer_ID'] = kmer_results_df.index + 1
kmer_results_df.head(10)

In [ ]:
analysisDF = kmer_results_df.sort_values(by='kmer_enr_mean',ascending=False)
analysisDF['kmer_rank'] = np.arange(len(analysisDF)) + 1
analysisDF.head(5)

In [ ]:
## export the results:
### V1: k-mers from N50C lib (5 input reads min, no Pcounts)
'''
#v1.1: comprehensive 4-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "kmer_results_all4mers_Enr_N50C" + ".csv"
kmer_results_df.to_csv(df_path)
#v1.1B: SORTED comprehensive 4-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "Sorted_kmer_results_all4mers_Enr_N50C" + ".csv"
analysisDF.to_csv(df_path)
'''

'''
#v1.2: comprehensive 5-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "kmer_results_all5mers_Enr_N50C" + ".csv"
kmer_results_df.to_csv(df_path)
#v1.2B: SORTED comprehensive 5-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "Sorted_kmer_results_all5mers_Enr_N50C" + ".csv"
analysisDF.to_csv(df_path)
'''

'''
#v1.3: comprehensive 6-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "kmer_results_all6mers_Enr_N50C" + ".csv"
kmer_results_df.to_csv(df_path)
#v1.3B: SORTED comprehensive 6-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "Sorted_kmer_results_all6mers_Enr_N50C" + ".csv"
analysisDF.to_csv(df_path)
'''

'''
#v1.4: comprehensive 7-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "kmer_results_all7mers_Enr_N50C" + ".csv"
kmer_results_df.to_csv(df_path)
#v1.4B: SORTED comprehensive 7-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "Sorted_kmer_results_all7mers_Enr_N50C" + ".csv"
analysisDF.to_csv(df_path)
'''

'''
#v1.5: comprehensive 8-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "kmer_results_all8mers_Enr_N50C" + ".csv"
kmer_results_df.to_csv(df_path)
#v1.5B: SORTED comprehensive 8-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "Sorted_kmer_results_all8mers_Enr_N50C" + ".csv"
analysisDF.to_csv(df_path)
'''



###V2: 6-mers from N50EPC lib, 5 input reads min, no PCounts
'''
#v2: comprehensive 6-mer results for growth selection enrichment, N50EPC lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "kmer_results_all6mers_Enr_N50EPC" + ".csv"
kmer_results_df.to_csv(df_path)
#v2B: SORTED comprehensive 6-mer results for growth selection enrichment, N50EPC lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "Sorted_kmer_results_all6mers_Enr_N50EPC" + ".csv"
analysisDF.to_csv(df_path)
'''


In [ ]:
## load (import) previously exported results: (used in subsequent steps, see below)

### V1: k-mers from N50C lib (5 input reads min, no Pcounts)
#v1: comprehensive 6-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "kmer_results_all6mers_Enr_N50C" + ".csv"
kmer_results_df = pandas.DataFrame.from_csv(df_path, index_col=None)
#v1B: SORTED comprehensive 6-mer results for growth selection enrichment, N50C lib, 5 input reads min, no PCounts
df_path = "/[[KMER_ANALYSIS_DIR]]/"\
+ "Sorted_kmer_results_all6mers_Enr_N50C" + ".csv"
analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)

### (or etc. as above for 4-8 mers, as appropriate)



### V2: 6-mers from N50EPC lib, 5 input reads min, no PCounts
'''
#load v2: comprehensive 6-mer results for growth selection enrichment, N50EPC lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "kmer_results_all6mers_Enr_N50EPC" + ".csv"
kmer_results_df = pandas.DataFrame.from_csv(df_path, index_col=None)
#load v2B: SORTED comprehensive 6-mer results for growth selection enrichment, N50EPC lib, 5 input reads min, no PCounts
df_path = "[[KMER_ANALYSIS_DIR]]/"\
+ "Sorted_kmer_results_all6mers_Enr_N50EPC" + ".csv"
analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)
'''



In [ ]:
## Plot k-mer analysis results

wtCYC1_Enr_N50C = N50C_df[N50C_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]
wtCYC1_Enr_N50EPC = N50EPC_enr_df[N50EPC_enr_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]

allData_df_list = ['N50C_df', 'N50EPC_enr_df', 'N50EPC_enr_df_zoomIn']

for allData_df_name in allData_df_list :
    
    if (allData_df_name == 'N50C_df'):
        allData_df = N50C_df
        figName = "kmer_results_all6mers_Enr__from_N50C" 
        df_path = "[[KMER_ANALYSIS_DIR]]/"\
        + "Sorted_kmer_results_all6mers_Enr_N50C" + ".csv"
        analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)
        CYC1_Enr = wtCYC1_Enr_N50C
        
    elif ((allData_df_name == 'N50EPC_enr_df') | (allData_df_name == 'N50EPC_enr_df_zoomIn')):
        allData_df = N50EPC_enr_df
        figName = "kmer_results_all6mers_Enr__from_N50EPC"
        df_path = "[[KMER_ANALYSIS_DIR]]/"\
        + "Sorted_kmer_results_all6mers_Enr_N50EPC" + ".csv"
        analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)
        CYC1_Enr = wtCYC1_Enr_N50EPC
    
    if (allData_df_name == 'N50EPC_enr_df_zoomIn'):
        figName = "kmer_results_all6mers_Enr__from_N50EPC_zoomedIn"
    
    analysisDF['kmer_rank'] = np.arange(len(analysisDF)) + 1

    kmerRank = analysisDF['kmer_rank'].tolist()

    kmer_mean = analysisDF['kmer_enr_mean'].tolist()

    kmer_sem = analysisDF['kmer_enr_sem'].tolist()

    NOkmer_mean = analysisDF['NOkmer_enr_mean'].tolist()

    NOkmer_sem = analysisDF['NOkmer_enr_sem'].tolist()

    plt.errorbar(kmerRank, kmer_mean, yerr = kmer_sem, capsize=3, alpha=1, fmt='.', markersize = 5, elinewidth = 1)
    plt.errorbar(kmerRank, NOkmer_mean, yerr = NOkmer_sem, capsize=3, color ='green', alpha=1, fmt='.',\
                 markersize = 5, elinewidth = 1)
    plt.hlines(y=np.mean(allData_df['log2_score']),xmin=-100+np.min(analysisDF['kmer_rank'])\
               ,xmax=1.1*np.max(analysisDF['kmer_rank']),color='red',alpha=1,linewidth=2)
    if(allData_df_name != 'N50EPC_enr_df_zoomIn') :
        plt.hlines(y=CYC1_Enr,xmin=-100+np.min(analysisDF['kmer_rank'])\
                   ,xmax=1.1*np.max(analysisDF['kmer_rank']),color='orange',alpha=1,linewidth=2)
    plt.xlabel('kmer_rank')
    plt.ylabel('mean enrichment of library seqs +- kmer (+-sem)')
    plt.title('Mean enrichment of library seqs containing kmer (blue)\nor lacking kmer (green); '+\
              'red = <Enr>all; orange = CYC1 Enr')
    #plt.xlim(0,2)
    if(allData_df_name != 'N50EPC_enr_df_zoomIn') :
        plt.ylim(-1.25,2.75)  ## for matching N50C, N50EPC library scales

    plt.show()


In [ ]:
## Plot results for 4-to-8-mer k-mer analyses

kmer_n_list = [4, 5, 6, 7, 8]    # set of kmer effects to plot
ymin = -1.1; ymax = 3.1          # setting y-scale for plots

wtCYC1_Enr_N50C = N50C_df[N50C_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]
wtCYC1_Enr_N50EPC = N50EPC_enr_df[N50EPC_enr_df['UTR']==CYC1_wt_N50]['log2_score'].iloc[0]

for kmer_n in kmer_n_list :
    CYC1_Enr = wtCYC1_Enr_N50C
    # comprehensive k-mer results for growth selection enrichment, N50C lib, 5 input reads min, no pseudocounts:
    if kmer_n == 4 :
        figName = "kmer_results_all4mers_Enr__from_N50C"
        df_path = "[[KMER_ANALYSIS_DIR]]/"\
        + "Sorted_kmer_results_all4mers_Enr_N50C" + ".csv"
        analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)
    if kmer_n == 5 :
        figName = "kmer_results_all5mers_Enr__from_N50C"
        df_path = "[[KMER_ANALYSIS_DIR]]/"\
        + "Sorted_kmer_results_all5mers_Enr_N50C" + ".csv"
        analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)
    if kmer_n == 6 :
        figName = "kmer_results_all6mers_Enr__from_N50C"
        df_path = "[[KMER_ANALYSIS_DIR]]/"\
        + "Sorted_kmer_results_all6mers_Enr_N50C" + ".csv"
        analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)
    if kmer_n == 7 :
        figName = "kmer_results_all7mers_Enr__from_N50C"
        df_path = "[[KMER_ANALYSIS_DIR]]/"\
        + "Sorted_kmer_results_all7mers_Enr_N50C" + ".csv"
        analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)
    if kmer_n == 8 :
        figName = "kmer_results_all8mers_Enr__from_N50C"
        df_path = "[[KMER_ANALYSIS_DIR]]/"\
        + "Sorted_kmer_results_all8mers_Enr_N50C" + ".csv"
        analysisDF = pandas.DataFrame.from_csv(df_path, index_col=None)

    allData_df = enrichment_df

    #############################

    analysisDF['kmer_rank'] = np.arange(len(analysisDF)) + 1

    kmerRank = analysisDF['kmer_rank'].tolist()

    kmer_mean = analysisDF['kmer_enr_mean'].tolist()

    kmer_sem = analysisDF['kmer_enr_sem'].tolist()

    NOkmer_mean = analysisDF['NOkmer_enr_mean'].tolist()

    NOkmer_sem = analysisDF['NOkmer_enr_sem'].tolist()

    plt.errorbar(kmerRank, kmer_mean, yerr = kmer_sem, capsize=3, alpha=1, fmt='.', markersize = 5, elinewidth = 1)
    plt.errorbar(kmerRank, NOkmer_mean, yerr = NOkmer_sem, capsize=3, color ='green', alpha=1, fmt='.',\
                 markersize = 5, elinewidth = 1)
    plt.hlines(y=np.mean(allData_df['log2_score']),xmin=-100+np.min(analysisDF['kmer_rank'])\
               ,xmax=1.1*np.max(analysisDF['kmer_rank']),color='red',alpha=1,linewidth=2)
    plt.hlines(y=CYC1_Enr,xmin=-100+np.min(analysisDF['kmer_rank'])\
               ,xmax=1.1*np.max(analysisDF['kmer_rank']),color='orange',alpha=1,linewidth=2)
    plt.xlabel('kmer_rank')
    plt.ylabel('mean enrichment of library seqs +- ' +str(kmer_n)+ 'mer (+-sem)')
    plt.title('Mean enrichment of library seqs containing ' +str(kmer_n)+\
              'mer (blue)\nor lacking ' +str(kmer_n)+ 'mer (green); red = <Enr>all; orange = CYC1 Enr')
    
    if(kmer_n==4):
        plt.xlim(-25,275)
    plt.ylim(ymin,ymax)  ## for matching scales across all plots

    plt.tight_layout()

    plt.show()


In [ ]:
## Generate histograms of enrichment (Enr) for 3' UTRs +/- each motif, N50C library 
## PLUS overlapping histograms of 3' UTR variants containing shuffles/control sequences for each motif.
## Also generate corresponding motif statistics dataframe.

import re

def sem(df):
    return np.std(df) / np.sqrt(df.shape[0])

def print_parameters(analysisDF,name):
    print
    print name + ':'
    print 'N = ' + str(analysisDF.shape[0])
    print 'mean +/- sem = ' + str(np.mean(analysisDF['log2_score'])) + ' +\- ' + str(sem(analysisDF['log2_score']))
    
def make_shuffles(motif_input,numShuffles_input):
    
    motif = motif_input
    numShuffles = numShuffles_input
    
    minDist = len(motif)/2
    print minDist

    shuffleList = []
    shuffleList.append(motif)
    #print shuffleList

    for i in range (0,numShuffles): #(0,5):
        continueLoop = True
        while(continueLoop == True):
            shuffleX = motif
            #print shuffleX
            shuffleX_list = list(shuffleX)
            random.shuffle(shuffleX_list)
            shuffleX = ''.join(shuffleX_list)
            #print shuffleX
            dist = lev.hamming(shuffleX, motif)
            if ((dist >= minDist) & (shuffleX not in shuffleList)) :
                #print "success"
                shuffleList.append(shuffleX)
                continueLoop = False
            #else:
                #print "fail, try again"

    print;print; print shuffleList
    return shuffleList
    
#set up histogram bins
bins = np.linspace(-10,10,40)

plotDF = N50C_df

print_parameters(plotDF, 'N50C, ALL')
analysisN = plotDF['log2_score'].shape[0]
plt.hist(plotDF['log2_score'], bins, alpha=0.5, color = 'grey', edgecolor='black',\
    density=True, label='N50-C, all variants'+ ',\nN = ' + str(analysisN) )
plt.legend(loc='upper left',prop={'size':7})
plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
#plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
plt.tight_layout()

#uncomment the below and add appropriate file directory in order to save figures:
'''
figName = 'N50C_+vs-motif-hists_'+'ALL variants'+'_log2E_hist'
output_path = "[[FIGURES_OUTPUT_DIR]]/"\
+ figName + '.png'
plt.savefig(output_path, dpi=1200)
output_path = "[[FIGURES_OUTPUT_DIR]]/"\
+ figName + '.pdf'
plt.savefig(output_path, dpi=1200)
'''

plt.show()

motifs_list = ['TATATA', 'TTTTTATA', 'GCGCGC', 'AATAAA|AAAAAA', 'TAAT...TAAT', 'TGTA.ATA', 'TGTA.A.TA', 'TGTA....TA',\
              'TTGT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT',\
               'AATATA', 'TTTATA', 'TAAATA', 'TATTTA', 'TATAAA', 'TATATT',\
              'CATATA', 'TGTATA', 'TACATA', 'TATGTA', 'TATACA', 'TATATG',\
              'ATTTTATA','TATTTATA','TTATTATA','TTTATATA','TTTTAATA','TTTTTTTA','TTTTTAAA','TTTTTATT',\
               'ATATAT','TTATAT','AAATAT','ATTTAT','ATAAAT','ATATTT','ATATAA',\
               'TATATATA','AATATATA','TTTATATA','TAAATATA','TATTTATA','TATAAATA','TATATTTA','TATATAAA','TATATATT',\
               'TATATATATA','AATATATATA','TTTATATATA','TAAATATATA','TATTTATATA','TATAAATATA',\
               'TATATTTATA','TATATAAATA','TATATATTTA','TATATATAAA','TATATATATT']

motif_names_list = ['(UA)3 EE', 'U5AUA EE', '(GC)3 control', 'AAWAAA PE', 'Puf1&2_site', 'Puf3_site', 'Puf4_site',\
                   'Puf5_site', 'Puf6_site', 'polyU_8X_control1(U4A4)', 'polyU_8X_control2(A4U4)',\
                    'polyU_8X_control3(A8)', 'polyU_8X_control4(U2A3U3)', 'polyU_8X_first25nt', 'polyU_8X_last25nt',\
                   'polyU_8X_control5(AU%)', 'AATATA', 'TTTATA', 'TAAATA', 'TATTTA', 'TATAAA', 'TATATT',\
              'CATATA', 'TGTATA', 'TACATA', 'TATGTA', 'TATACA', 'TATATG',\
              'ATTTTATA','TATTTATA','TTATTATA','TTTATATA','TTTTAATA','TTTTTTTA','TTTTTAAA','TTTTTATT',\
                   'ATATAT','TTATAT','AAATAT','ATTTAT','ATAAAT','ATATTT','ATATAA',\
               'TATATATA','AATATATA','TTTATATA','TAAATATA','TATTTATA','TATAAATA','TATATTTA','TATATAAA','TATATATT',\
               'TATATATATA','AATATATATA','TTTATATATA','TAAATATATA','TATTTATATA','TATAAATATA',\
               'TATATTTATA','TATATAAATA','TATATATTTA','TATATATAAA','TATATATATT']

motif_stats_names = []
motif_stats_means = []
motif_stats_sems = []
motif_stats_N = []
motif_stats_tTest_pVal_withMotif_VS_withoutMotif = []
motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle = []
motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle = []

motif_stats_names.append('N50C ALL')
motif_stats_means.append(np.mean(plotDF['log2_score']))
motif_stats_N.append(plotDF['log2_score'].shape[0])
motif_stats_sems.append(sem(plotDF['log2_score']))
motif_stats_tTest_pVal_withMotif_VS_withoutMotif.append(np.nan)
motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle.append(np.nan)
motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle.append(np.nan)


for i in range(0,len(motifs_list)) :
    plotDF = N50C_df
    motif = motifs_list[i]
    motif_name = motif_names_list[i]
    
    if(motif=='TATATA') :
        numShuffles = 10
        shuffleList = make_shuffles(motif,numShuffles)

    elif(motif=='TTTTTATA') :
        numShuffles = 15
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='GCGCGC') :
        numShuffles = 10
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='AATAAA|AAAAAA') :
        numShuffles = -1
        control_motif = 'AAATTT'
        
    elif(motif=='TAAT...TAAT') :
        numShuffles = 50
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='TGTA.ATA') :
        numShuffles = 50
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='TGTA.A.TA') :
        numShuffles = 50
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='TGTA....TA') :
        numShuffles = 50
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='TTGT') :
        numShuffles = 3
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif_name=='polyU_8X_control1(U4A4)') :
        numShuffles = -1
        control_motif = 'TTTTAAAA'
        
    elif(motif_name=='polyU_8X_control2(A4U4)') :
        numShuffles = -1
        control_motif = 'AAAATTTT'
             
    elif(motif_name=='polyU_8X_control3(A8)') :
        numShuffles = -1
        control_motif = 'AAAAAAAA'
        
    elif(motif_name=='polyU_8X_control4(U2A3U3)') :
        numShuffles = -1
        control_motif = 'TTAAATTT'
        
    elif(motif_name=='polyU_8X_first25nt') :
        numShuffles = -2
        
    elif(motif_name=='polyU_8X_last25nt') :
        numShuffles = -3
        
    elif(motif_name=='polyU_8X_control5(AU%)') :
        numShuffles = -4
        
    elif(i in range(16,27+1)) :
        numShuffles = -1
        control_motif = 'AAATTT'
        
    elif(i in range(28,35+1)) :
        numShuffles = -1
        control_motif = 'AAAATTTT'
        
    elif(i in range(36,42+1)) :
        numShuffles = -5
        control_motif = 'AAATTT'
        
    elif(i in range(43,51+1)) :
        numShuffles = -1
        control_motif = 'AAAATTTT'
        
    elif(i in range(52,62+1)) :
        numShuffles = -1
        control_motif = 'AAAAATTTTT'
        
    simple_motif_name = motif_name
    if('polyU_8X' in motif_name) :
        simple_motif_name = 'polyU_8X'

    analysisDF = plotDF[plotDF['UTR'].str.contains(motif)==False]
    print_parameters(analysisDF,'N50C, non-' + simple_motif_name + '-containing')
    analysisN = analysisDF['log2_score'].shape[0]
    plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'blue', edgecolor='black',\
             density=True, label='N50-C *not* cont. ' + simple_motif_name + ',\nN = '+ str(analysisN))
    plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
    plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
    #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
    motif_stats_names.append(simple_motif_name+'_NOmotif')
    motif_stats_means.append(np.mean(analysisDF['log2_score']))
    motif_stats_N.append(analysisDF['log2_score'].shape[0])
    motif_stats_sems.append(sem(analysisDF['log2_score']))
    ptest_DF_noMotif = analysisDF
    
    if (numShuffles == -5) : 
        analysisDF = plotDF[(plotDF['UTR'].str.contains('TATATA')==False)\
                                                & (enrichment_df['UTR'].str.contains(motif))]
        print_parameters(analysisDF,'N50C, ' + simple_motif_name +' -containing (no TATATA)')
        analysisN = analysisDF['log2_score'].shape[0]
        plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'red', edgecolor='black',\
                 density=True, label='N50-C cont. ' + simple_motif_name + ' (no TATATA), N = '+ str(analysisN))
        plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_+motif')
        motif_stats_means.append(np.mean(analysisDF['log2_score']))
        motif_stats_N.append(analysisDF['log2_score'].shape[0])
        motif_stats_sems.append(sem(analysisDF['log2_score']))
        ptest_DF_withMotif = analysisDF
    else:
        analysisDF = plotDF[plotDF['UTR'].str.contains(motif)]
        print_parameters(analysisDF,'N50C, ' + simple_motif_name +' -containing')
        analysisN = analysisDF['log2_score'].shape[0]
        plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'red', edgecolor='black',\
                 density=True, label='N50-C cont. ' + simple_motif_name + ',\nN = '+ str(analysisN))
        plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_+motif')
        motif_stats_means.append(np.mean(analysisDF['log2_score']))
        motif_stats_N.append(analysisDF['log2_score'].shape[0])
        motif_stats_sems.append(sem(analysisDF['log2_score']))
        ptest_DF_withMotif = analysisDF
    
    if (numShuffles == -1) :
        analysisDF = plotDF[(plotDF['UTR'].str.contains(motif)==False)\
                                                & (enrichment_df['UTR'].str.contains(control_motif))]
        print_parameters(analysisDF,'N50C, ' + control_motif +' control')
        analysisN = analysisDF['log2_score'].shape[0]
        plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='N50-C cont. ' + control_motif + ' control' + ',\nN = '+ str(analysisN))
        plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(motif_name+'_control')
        motif_stats_means.append(np.mean(analysisDF['log2_score'])) 
        motif_stats_N.append(analysisDF['log2_score'].shape[0])
        motif_stats_sems.append(sem(analysisDF['log2_score']))
        ptest_DF_control = analysisDF
    elif (numShuffles == -2) :
        analysisDF = plotDF[(plotDF['UTR'].str[1:25].str.contains(motif))]
        print_parameters(analysisDF,'N50C, ' + simple_motif_name +' in first 25 nt')
        analysisN = analysisDF['log2_score'].shape[0]
        plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='N50-C cont. ' + simple_motif_name +' in first 25 nt' + ',\nN = '+ str(analysisN))
        plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_first25nt')
        motif_stats_means.append(np.mean(analysisDF['log2_score'])) 
        motif_stats_N.append(analysisDF['log2_score'].shape[0])
        motif_stats_sems.append(sem(analysisDF['log2_score']))
        ptest_DF_control = analysisDF
    elif (numShuffles == -3) :
        analysisDF = plotDF[(plotDF['UTR'].str[26:50].str.contains(motif))]
        print_parameters(analysisDF,'N50C, ' + simple_motif_name +' in last 25 nt')
        analysisN = analysisDF['log2_score'].shape[0]
        plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='N50-C cont. ' + simple_motif_name +' in last 25 nt' + ',\nN = '+ str(analysisN))
        plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_last25nt')
        motif_stats_means.append(np.mean(analysisDF['log2_score'])) 
        motif_stats_N.append(analysisDF['log2_score'].shape[0])
        motif_stats_sems.append(sem(analysisDF['log2_score']))
        ptest_DF_control = analysisDF
    elif (numShuffles == -5) :
        analysisDF = plotDF[(plotDF['UTR'].str.contains(motif)==False)\
                            & (plotDF['UTR'].str.contains('TATATA')==False)\
                            & (enrichment_df['UTR'].str.contains(control_motif))]
        print_parameters(analysisDF,'N50C, ' + control_motif +' control (no TATATA)')
        analysisN = analysisDF['log2_score'].shape[0]
        plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='N50-C cont. ' + control_motif + ' control (no TATATA)' + ',\nN = '+ str(analysisN))
        plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_control')
        motif_stats_means.append(np.mean(analysisDF['log2_score'])) 
        motif_stats_N.append(analysisDF['log2_score'].shape[0])
        motif_stats_sems.append(sem(analysisDF['log2_score']))
        ptest_DF_control = analysisDF
    elif (numShuffles == -4) :
        
        ### make T4A4 shuffles: 8-mer %AU control ########
        shuffles_motif = 'TTTTAAAA' # T4A4 %AU shuffles control generation (4 U's + 4 A's in various shuffled configurations)
        minDist = len(shuffles_motif)/2  ## this is in addition to filter for 'TATA' content (see below)

        numShuffles = 35

        shuffleList = []
        shuffleList.append(shuffles_motif)
        #print shuffleList

        for i in range (0,numShuffles): #(0,5):
            continueLoop = True
            while(continueLoop == True):
                shuffleX = shuffles_motif
                #print shuffleX
                shuffleX_list = list(shuffleX)
                random.shuffle(shuffleX_list)
                shuffleX = ''.join(shuffleX_list)
                #print shuffleX
                dist = lev.hamming(shuffleX, shuffles_motif)
                if ((dist >= minDist) & (shuffleX not in shuffleList) & (('TATA' in shuffleX) == False)) :
                    #print "success"
                    shuffleList.append(shuffleX)
                    continueLoop = False
                #else:
                    #print "fail, try again"

        print;print; print shuffleList
        ##############################################
        
        shuffles = shuffleList  #want to include T4A4 starting sequence in this case
        analysisDF = plotDF[(plotDF['UTR'].str.contains(motif)==False)\
                                                & (plotDF['UTR'].str.contains('|'.join(shuffles)))]
        print_parameters(analysisDF,'N50C, ' + '%AU (4xA,4xU) control')
        analysisN = analysisDF['log2_score'].shape[0]
        plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='N50-C cont. ' + '%AU (4xA,4xU) control' + ',\nN = '+ str(analysisN))
        plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_%AU(4xA,4xU)control')
        motif_stats_means.append(np.mean(analysisDF['log2_score']))
        motif_stats_N.append(analysisDF['log2_score'].shape[0])
        motif_stats_sems.append(sem(analysisDF['log2_score']))
        ptest_DF_control = analysisDF
    else :
        shuffles = shuffleList[1:]  #exclude the consensus motif from the list of shuffled sequences
        analysisDF = plotDF[(plotDF['UTR'].str.contains(motif)==False)\
                                                & (plotDF['UTR'].str.contains('|'.join(shuffles)))]
        print_parameters(analysisDF,'N50C, ' + simple_motif_name +' shuffles')
        analysisN = analysisDF['log2_score'].shape[0]
        plt.hist(analysisDF['log2_score'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='N50-C cont. ' + simple_motif_name + ' shuffles' + ',\nN = '+ str(analysisN))
        plt.xlabel('enrichment', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('N50-C library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_shuffles')
        motif_stats_means.append(np.mean(analysisDF['log2_score']))
        motif_stats_N.append(analysisDF['log2_score'].shape[0])
        motif_stats_sems.append(sem(analysisDF['log2_score']))
        ptest_DF_control = analysisDF

    plt.legend(loc='upper left',prop={'size':7})
    plt.tight_layout()
    
    ### perform T-tests and save p-values #####
    Ttest = scipy.stats.ttest_ind(ptest_DF_withMotif['log2_score'],ptest_DF_noMotif['log2_score'],\
                          equal_var=False,nan_policy='propagate')
    pVal = Ttest[1]   #;   print pVal 
    for i in range(0,3) :  #must add to list 3 times because of the table structure (3 entries per motif)
        motif_stats_tTest_pVal_withMotif_VS_withoutMotif.append(pVal)

    Ttest = scipy.stats.ttest_ind(ptest_DF_withMotif['log2_score'],ptest_DF_control['log2_score'],\
                          equal_var=False,nan_policy='propagate')
    pVal = Ttest[1]   #;   print pVal 
    for i in range(0,3) :  #must add to list 3 times because of the table structure (3 entries per motif)
        motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle.append(pVal)
    
    Ttest = scipy.stats.ttest_ind(ptest_DF_noMotif['log2_score'],ptest_DF_control['log2_score'],\
                          equal_var=False,nan_policy='propagate')
    pVal = Ttest[1]   #;   print pVal 
    for i in range(0,3) :  #must add to list 3 times because of the table structure (3 entries per motif)
        motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle.append(pVal)
    #############################################
    
    # uncomment the below to save the plots (add desired file directory)
    '''
    figName = 'N50C_+vs-motif-hists_'+motif_name+'_log2E_hist'
    output_path = "[[FIGURES_OUTPUT_DIR]]/"\
    + figName + '.png'
    plt.savefig(output_path, dpi=1200)
    output_path = "[[FIGURES_OUTPUT_DIR]]/"\
    + figName + '.pdf'
    plt.savefig(output_path, dpi=1200)
    '''
    
    plt.show()  ## optional, if want to see all plots
    #plt.close()  ## alternative, if only want to save all plots
    

print motif_stats_names; print motif_stats_means; print motif_stats_sems; print motif_stats_N;\
print motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle; print motif_stats_tTest_pVal_withMotif_VS_withoutMotif;\
print motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle

print; print len(motif_stats_tTest_pVal_withMotif_VS_withoutMotif);\
print len(motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle);\
print len(motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle)

N50C_motif_stats_df = pandas.DataFrame()

N50C_motif_stats_df['motif_stats_names'] = motif_stats_names
N50C_motif_stats_df['motif_stats_means'] = motif_stats_means
N50C_motif_stats_df['motif_stats_sems'] = motif_stats_sems
N50C_motif_stats_df['motif_stats_N'] = motif_stats_N
N50C_motif_stats_df['motif_stats_tTest_pVal_withMotif_VS_withoutMotif']\
= motif_stats_tTest_pVal_withMotif_VS_withoutMotif
N50C_motif_stats_df['motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle']\
= motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle
N50C_motif_stats_df['motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle']\
= motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle

print N50C_motif_stats_df.head(5)


In [ ]:
## export motif stats dataframe (for enrichment in N50C library) as CSV file
df_path = "[[OUTPUT_DIR_MOTIF_STATS]]/"\
+ "Motif_Enr_Stats_from_N50C" + ".csv"
N50C_motif_stats_df.to_csv(df_path)


In [ ]:
## Generate histograms of mRNA half-lives (Geisberg et al. 2014) for native yeast 3' UTRs +/- each motif
## PLUS overlapping histograms of 3' UTR variants containing shuffles/control sequences for each motif.
## Also generate corresponding motif statistics dataframe.

import re

def sem(df):
    return np.std(df) / np.sqrt(df.shape[0])

def print_parameters(analysisDF,name):
    print
    print name + ':'
    print 'N = ' + str(analysisDF.shape[0])
    print 'mean +/- sem = ' + str(np.mean(analysisDF['Geisberg/min'])) + ' +\- ' + str(sem(analysisDF['Geisberg/min']))
    
def make_shuffles(motif_input,numShuffles_input):
    
    motif = motif_input
    numShuffles = numShuffles_input
    
    minDist = len(motif)/2
    print minDist

    shuffleList = []
    shuffleList.append(motif)
    #print shuffleList

    for i in range (0,numShuffles): #(0,5):
        continueLoop = True
        while(continueLoop == True):
            shuffleX = motif
            #print shuffleX
            shuffleX_list = list(shuffleX)
            random.shuffle(shuffleX_list)
            shuffleX = ''.join(shuffleX_list)
            #print shuffleX
            dist = lev.hamming(shuffleX, motif)
            if ((dist >= minDist) & (shuffleX not in shuffleList)) :
                #print "success"
                shuffleList.append(shuffleX)
                continueLoop = False
            #else:
                #print "fail, try again"

    print;print; print shuffleList
    return shuffleList

    
#set up histogram bins
bins = np.linspace(0,150,50)

plotDF = nativeUTRs_df

print_parameters(plotDF, 'native, ALL')
analysisN = plotDF['Geisberg/min'].shape[0]
plt.hist(plotDF['Geisberg/min'], bins, alpha=0.5, color = 'grey', edgecolor='black',\
    density=True, label='native, all variants'+ ',\nN = ' + str(analysisN) )
plt.legend(loc='upper right',prop={'size':7})
plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
#plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
plt.tight_layout()

# uncomment the below to save figures (add desired output filepath)
'''
figName = 'native_+vs-motif-hists_'+'ALL variants'+'_halfLife(Geisberg)_hist'
output_path = "[[FIGURES_OUTPUT_DIRECTORY]]/"\
+ figName + '.png'
plt.savefig(output_path, dpi=1200)
output_path = "[[FIGURES_OUTPUT_DIRECTORY]]/"\
+ figName + '.pdf'
plt.savefig(output_path, dpi=1200)
'''

plt.show()

motifs_list = ['TATATA', 'TTTTTATA', 'GCGCGC', 'AATAAA|AAAAAA', 'TAAT...TAAT', 'TGTA.ATA', 'TGTA.A.TA', 'TGTA....TA',\
              'TTGT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT', 'TTTTTTTT',\
               'AATATA', 'TTTATA', 'TAAATA', 'TATTTA', 'TATAAA', 'TATATT',\
              'CATATA', 'TGTATA', 'TACATA', 'TATGTA', 'TATACA', 'TATATG',\
              'ATTTTATA','TATTTATA','TTATTATA','TTTATATA','TTTTAATA','TTTTTTTA','TTTTTAAA','TTTTTATT',\
               'ATATAT','TTATAT','AAATAT','ATTTAT','ATAAAT','ATATTT','ATATAA',\
               'TATATATA','AATATATA','TTTATATA','TAAATATA','TATTTATA','TATAAATA','TATATTTA','TATATAAA','TATATATT',\
               'TATATATATA','AATATATATA','TTTATATATA','TAAATATATA','TATTTATATA','TATAAATATA',\
               'TATATTTATA','TATATAAATA','TATATATTTA','TATATATAAA','TATATATATT']

motif_names_list = ['(UA)3 EE', 'U5AUA EE', '(GC)3 control', 'AAWAAA PE', 'Puf1&2_site', 'Puf3_site', 'Puf4_site',\
                   'Puf5_site', 'Puf6_site', 'polyU_8X_control1(U4A4)', 'polyU_8X_control2(A4U4)',\
                    'polyU_8X_control3(A8)', 'polyU_8X_control4(U2A3U3)', 'polyU_8X_first25nt', 'polyU_8X_last25nt',\
                   'polyU_8X_control5(AU%)', 'AATATA', 'TTTATA', 'TAAATA', 'TATTTA', 'TATAAA', 'TATATT',\
              'CATATA', 'TGTATA', 'TACATA', 'TATGTA', 'TATACA', 'TATATG',\
              'ATTTTATA','TATTTATA','TTATTATA','TTTATATA','TTTTAATA','TTTTTTTA','TTTTTAAA','TTTTTATT',\
                   'ATATAT','TTATAT','AAATAT','ATTTAT','ATAAAT','ATATTT','ATATAA',\
               'TATATATA','AATATATA','TTTATATA','TAAATATA','TATTTATA','TATAAATA','TATATTTA','TATATAAA','TATATATT',\
               'TATATATATA','AATATATATA','TTTATATATA','TAAATATATA','TATTTATATA','TATAAATATA',\
               'TATATTTATA','TATATAAATA','TATATATTTA','TATATATAAA','TATATATATT']

motif_stats_names = []
motif_stats_means = []
motif_stats_sems = []
motif_stats_N = []
motif_stats_tTest_pVal_withMotif_VS_withoutMotif = []
motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle = []
motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle = []

motif_stats_names.append('native ALL')
motif_stats_means.append(np.mean(plotDF['Geisberg/min']))
motif_stats_N.append(plotDF['Geisberg/min'].shape[0])
motif_stats_sems.append(sem(plotDF['Geisberg/min']))
motif_stats_tTest_pVal_withMotif_VS_withoutMotif.append(np.nan)
motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle.append(np.nan)
motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle.append(np.nan)


for i in range(0,len(motifs_list)) :
    plotDF = nativeUTRs_df
    motif = motifs_list[i]
    motif_name = motif_names_list[i]
    
    if(motif=='TATATA') :
        numShuffles = 10
        shuffleList = make_shuffles(motif,numShuffles)

    elif(motif=='TTTTTATA') :
        numShuffles = 15
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='GCGCGC') :
        numShuffles = 10
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='AATAAA|AAAAAA') :
        numShuffles = -1
        control_motif = 'AAATTT'
        
    elif(motif=='TAAT...TAAT') :
        numShuffles = 50
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='TGTA.ATA') :
        numShuffles = 50
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='TGTA.A.TA') :
        numShuffles = 50
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='TGTA....TA') :
        numShuffles = 50
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif=='TTGT') :
        numShuffles = 3
        shuffleList = make_shuffles(motif,numShuffles)
        
    elif(motif_name=='polyU_8X_control1(U4A4)') :
        numShuffles = -1
        control_motif = 'TTTTAAAA'
        
    elif(motif_name=='polyU_8X_control2(A4U4)') :
        numShuffles = -1
        control_motif = 'AAAATTTT'
             
    elif(motif_name=='polyU_8X_control3(A8)') :
        numShuffles = -1
        control_motif = 'AAAAAAAA'
        
    elif(motif_name=='polyU_8X_control4(U2A3U3)') :
        numShuffles = -1
        control_motif = 'TTAAATTT'
        
    elif(motif_name=='polyU_8X_first25nt') :
        numShuffles = -2
        
    elif(motif_name=='polyU_8X_last25nt') :
        numShuffles = -3
        
    elif(motif_name=='polyU_8X_control5(AU%)') :
        numShuffles = -4
        
    elif(i in range(16,27+1)) :
        numShuffles = -1
        control_motif = 'AAATTT'
        
    elif(i in range(28,35+1)) :
        numShuffles = -1
        control_motif = 'AAAATTTT'
        
    elif(i in range(36,42+1)) :
        numShuffles = -5
        control_motif = 'AAATTT'
        
    elif(i in range(43,51+1)) :
        numShuffles = -1
        control_motif = 'AAAATTTT'
        
    elif(i in range(52,62+1)) :
        numShuffles = -1
        control_motif = 'AAAAATTTTT'
        

    simple_motif_name = motif_name
    if('polyU_8X' in motif_name) :
        simple_motif_name = 'polyU_8X'


    analysisDF = plotDF[plotDF['UTR'].str.contains(motif)==False]
    print_parameters(analysisDF,'native, non-' + simple_motif_name + '-containing')
    analysisN = analysisDF['Geisberg/min'].shape[0]
    plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'blue', edgecolor='black',\
             density=True, label='native *not* cont. ' + simple_motif_name + ',\nN = '+ str(analysisN))
    plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
    plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
    #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
    motif_stats_names.append(simple_motif_name+'_NOmotif')
    motif_stats_means.append(np.mean(analysisDF['Geisberg/min']))
    motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
    motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
    ptest_DF_noMotif = analysisDF
    
    if (numShuffles == -5) : 
        analysisDF = plotDF[(plotDF['UTR'].str.contains('TATATA')==False)\
                                                & (enrichment_df['UTR'].str.contains(motif))]
        print_parameters(analysisDF,'native, ' + simple_motif_name +' -containing (no TATATA)')
        analysisN = analysisDF['Geisberg/min'].shape[0]
        plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'red', edgecolor='black',\
                 density=True, label='native cont. ' + simple_motif_name + ' (no TATATA), N = '+ str(analysisN))
        plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_+motif')
        motif_stats_means.append(np.mean(analysisDF['Geisberg/min']))
        motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
        motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
        ptest_DF_withMotif = analysisDF
    else:
        analysisDF = plotDF[plotDF['UTR'].str.contains(motif)]
        print_parameters(analysisDF,'native, ' + simple_motif_name +' -containing')
        analysisN = analysisDF['Geisberg/min'].shape[0]
        plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'red', edgecolor='black',\
                 density=True, label='native cont. ' + simple_motif_name + ',\nN = '+ str(analysisN))
        plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_+motif')
        motif_stats_means.append(np.mean(analysisDF['Geisberg/min']))
        motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
        motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
        ptest_DF_withMotif = analysisDF
    
    if (numShuffles == -1) :
        analysisDF = plotDF[(plotDF['UTR'].str.contains(motif)==False)\
                                                & (enrichment_df['UTR'].str.contains(control_motif))]
        print_parameters(analysisDF,'native, ' + control_motif +' control')
        analysisN = analysisDF['Geisberg/min'].shape[0]
        plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='native cont. ' + control_motif + ' control' + ',\nN = '+ str(analysisN))
        plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(motif_name+'_control')
        motif_stats_means.append(np.mean(analysisDF['Geisberg/min'])) 
        motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
        motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
        ptest_DF_control = analysisDF
    elif (numShuffles == -2) :
        analysisDF = plotDF[(plotDF['UTR'].str[1:25].str.contains(motif))]
        print_parameters(analysisDF,'native, ' + simple_motif_name +' in first 25 nt')
        analysisN = analysisDF['Geisberg/min'].shape[0]
        plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='native cont. ' + simple_motif_name +' in first 25 nt' + ',\nN = '+ str(analysisN))
        plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_first25nt')
        motif_stats_means.append(np.mean(analysisDF['Geisberg/min'])) 
        motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
        motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
        ptest_DF_control = analysisDF
    elif (numShuffles == -3) :
        analysisDF = plotDF[(plotDF['UTR'].str[26:50].str.contains(motif))]
        print_parameters(analysisDF,'native, ' + simple_motif_name +' in last 25 nt')
        analysisN = analysisDF['Geisberg/min'].shape[0]
        plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='native cont. ' + simple_motif_name +' in last 25 nt' + ',\nN = '+ str(analysisN))
        plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_last25nt')
        motif_stats_means.append(np.mean(analysisDF['Geisberg/min'])) 
        motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
        motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
        ptest_DF_control = analysisDF
    elif (numShuffles == -5) :
        analysisDF = plotDF[(plotDF['UTR'].str.contains(motif)==False)\
                            & (plotDF['UTR'].str.contains('TATATA')==False)\
                            & (enrichment_df['UTR'].str.contains(control_motif))]
        print_parameters(analysisDF,'native, ' + control_motif +' control (no TATATA)')
        analysisN = analysisDF['Geisberg/min'].shape[0]
        plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='native cont. ' + control_motif + ' control (no TATATA)' + ',\nN = '+ str(analysisN))
        plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_control')
        motif_stats_means.append(np.mean(analysisDF['Geisberg/min'])) 
        motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
        motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
        ptest_DF_control = analysisDF
    elif (numShuffles == -4) :
        
        ### make T4A4 shuffles: 8-mer %AU control ########
        shuffles_motif = 'TTTTAAAA' # T4A4 %AU shuffles control generation (4 U's + 4 A's in various shuffled configurations)
        minDist = len(shuffles_motif)/2  ## this is in addition to filter for 'TATA' content (see below)

        numShuffles = 35

        shuffleList = []
        shuffleList.append(shuffles_motif)
        #print shuffleList

        for i in range (0,numShuffles): #(0,5):
            continueLoop = True
            while(continueLoop == True):
                shuffleX = shuffles_motif
                #print shuffleX
                shuffleX_list = list(shuffleX)
                random.shuffle(shuffleX_list)
                shuffleX = ''.join(shuffleX_list)
                #print shuffleX
                dist = lev.hamming(shuffleX, shuffles_motif)
                if ((dist >= minDist) & (shuffleX not in shuffleList) & (('TATA' in shuffleX) == False)) :
                    #print "success"
                    shuffleList.append(shuffleX)
                    continueLoop = False
                #else:
                    #print "fail, try again"

        print;print; print shuffleList
        ##############################################
        
        shuffles = shuffleList  #want to include T4A4 starting sequence in this case
        analysisDF = plotDF[(plotDF['UTR'].str.contains(motif)==False)\
                                                & (plotDF['UTR'].str.contains('|'.join(shuffles)))]
        print_parameters(analysisDF,'native, ' + '%AU (4xA,4xU) control')
        analysisN = analysisDF['Geisberg/min'].shape[0]
        plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='native cont. ' + '%AU (4xA,4xU) control' + ',\nN = '+ str(analysisN))
        plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_%AU(4xA,4xU)control')
        motif_stats_means.append(np.mean(analysisDF['Geisberg/min']))
        motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
        motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
        ptest_DF_control = analysisDF
    else :
        shuffles = shuffleList[1:]  #exclude the consensus motif from the list of shuffled sequences
        analysisDF = plotDF[(plotDF['UTR'].str.contains(motif)==False)\
                                                & (plotDF['UTR'].str.contains('|'.join(shuffles)))]
        print_parameters(analysisDF,'native, ' + simple_motif_name +' shuffles')
        analysisN = analysisDF['Geisberg/min'].shape[0]
        plt.hist(analysisDF['Geisberg/min'], bins, alpha=0.5, color = 'white', edgecolor='black',\
                 density=True, label='native cont. ' + simple_motif_name + ' shuffles' + ',\nN = '+ str(analysisN))
        plt.xlabel('half-life (1/min)', fontweight='bold', fontsize=14,labelpad=10)
        plt.ylabel('density', fontweight='bold', fontsize=14,labelpad=10)
        #plt.title('native library', fontweight='bold', fontsize=14,labelpad=10)
        motif_stats_names.append(simple_motif_name+'_shuffles')
        motif_stats_means.append(np.mean(analysisDF['Geisberg/min']))
        motif_stats_N.append(analysisDF['Geisberg/min'].shape[0])
        motif_stats_sems.append(sem(analysisDF['Geisberg/min']))
        ptest_DF_control = analysisDF

    plt.legend(loc='upper right',prop={'size':7})
    plt.tight_layout()
    
    ### perform T-tests and save p-values #####
    Ttest = scipy.stats.ttest_ind(ptest_DF_withMotif['Geisberg/min'],ptest_DF_noMotif['Geisberg/min'],\
                          equal_var=False,nan_policy='propagate')
    pVal = Ttest[1]   #;   print pVal 
    for i in range(0,3) :  #must add to list 3 times because of the table structure (3 entries per motif)
        motif_stats_tTest_pVal_withMotif_VS_withoutMotif.append(pVal)

    Ttest = scipy.stats.ttest_ind(ptest_DF_withMotif['Geisberg/min'],ptest_DF_control['Geisberg/min'],\
                          equal_var=False,nan_policy='propagate')
    pVal = Ttest[1]   #;   print pVal 
    for i in range(0,3) :  #must add to list 3 times because of the table structure (3 entries per motif)
        motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle.append(pVal)
    
    Ttest = scipy.stats.ttest_ind(ptest_DF_noMotif['Geisberg/min'],ptest_DF_control['Geisberg/min'],\
                          equal_var=False,nan_policy='propagate')
    pVal = Ttest[1]   #;   print pVal 
    for i in range(0,3) :  #must add to list 3 times because of the table structure (3 entries per motif)
        motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle.append(pVal)
    #############################################
    
    # uncomment the below to save figures (add desired output filepath)
    '''
    figName = 'native_+vs-motif-hists_'+motif_name+'_halfLife(Geisberg)_hist'
    output_path = "[[FIGURES_OUTPUT_DIRECTORY]]/"\
    + figName + '.png'
    plt.savefig(output_path, dpi=1200)
    output_path = "[[FIGURES_OUTPUT_DIRECTORY]]/"\
    + figName + '.pdf'
    plt.savefig(output_path, dpi=1200)
    '''
    
    plt.show()  ## optional, if want to see all plots
    #plt.close()  ## alternative, if only want to save all plots
    

print motif_stats_names; print motif_stats_means; print motif_stats_sems; print motif_stats_N;\
print motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle; print motif_stats_tTest_pVal_withMotif_VS_withoutMotif;\
print motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle

print; print len(motif_stats_tTest_pVal_withMotif_VS_withoutMotif);\
print len(motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle);\
print len(motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle)

native_motif_stats_df = pandas.DataFrame()

native_motif_stats_df['motif_stats_names'] = motif_stats_names
native_motif_stats_df['motif_stats_means'] = motif_stats_means
native_motif_stats_df['motif_stats_sems'] = motif_stats_sems
native_motif_stats_df['motif_stats_N'] = motif_stats_N
native_motif_stats_df['motif_stats_tTest_pVal_withMotif_VS_withoutMotif']\
= motif_stats_tTest_pVal_withMotif_VS_withoutMotif
native_motif_stats_df['motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle']\
= motif_stats_tTest_pVal_withMotif_VS_ControlOrShuffle
native_motif_stats_df['motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle']\
= motif_stats_tTest_pVal_withoutMotif_VS_ControlOrShuffle

print native_motif_stats_df.head(5)


In [ ]:
## export motif stats dataframe (for half-lives of native yeast mRNAs, from Geisberg et al. 2014) as CSV file
df_path = "[[MOTIF_STATS_OUTPUT_PATH]]/"\
+ "Motif_HalfLife_Stats_from_nativeUTRs_Geisberg" + ".csv"
native_motif_stats_df.to_csv(df_path)


In [ ]:
### Generate list of native 3' UTR sequences with half-lives

## merge native 3' UTR sequence with isoform half-life

#Supplementary information from Geisberg et al. 
RNAs = pandas.read_excel('geisberg_half-lives.xls')
isoforms = pandas.read_excel('geisberg_isoforms.xls')

#Native sequeneces from curated list of yeast genes with 1000 bp upstream and downstream of ORF from Saccharomyces Genome Database
seqs = pandas.read_csv('orf_genomic_1000_all_SGD_names.tsv', sep = '\t')
seqs = seqs.drop_duplicates(subset=['systematic name'], keep=False)


## merge dataframes
isoforms_seqs = pandas.merge(isoforms, seqs, how = "inner", on = 'systematic name')

isoforms_seqs["Sequence"] = isoforms_seqs["Sequence"].astype(str)
isoforms_seqs['truncated']= isoforms_seqs['Sequence'].str.slice(-1000,)
isoforms_seqs["Position relative to stop codon"] = isoforms_seqs["Position relative to stop codon"].fillna(0.0).astype(int)

isoforms_seqs['trimmed'] = isoforms_seqs.apply(lambda x: x['truncated'][:x['Position relative to stop codon']], axis=1)

del isoforms_seqs['Sequence']
del isoforms_seqs['truncated']

allseqs = pandas.merge(RNAs, isoforms_seqs, how = 'left', on = 'systematic name')
allseqs = allseqs.dropna(subset = ['trimmed'], inplace=True)

del allseqs['difference']
del allseqs['chrom_x']
del allseqs['chrom_y']
del allseqs['Description']
del allseqs['Absolute Peak Coordinate']
del allseqs['SimpleType']
del allseqs['common name_y']

allseqs.rename(columns={'trimmed': 'sequence', 'common name_x': 'common name', 'Half-Life           (in minutes)':'Gene Half-Life'}).to_csv('Geisberg2014_native-mRNA-half-life_with_sequence.csv')